In [ ]:
import os
import re
import json
import time
from tqdm import tqdm

In [ ]:
# MODEL 2: STRUKTUR PERATURAN DASAR DAN AMANDEMEN SAMA

class RegulationParser:

    def __init__(self):
        self.ALPHABET_TO_NUMERIC = {chr(i): f'{i - 64:02}' for i in range(65, 91)}
        self.WORD_TO_NUMBER = {
            "kesatu": 1, "kedua": 2, "ketiga": 3, "keempat": 4, "kelima": 5,
            "keenam": 6, "ketujuh": 7, "kedelapan": 8, "kesembilan": 9, "kesepuluh": 10,
            "kesebelas": 11, "kedua belas": 12, "ketiga belas": 13, "keempat belas": 14, "kelima belas": 15,
            "keenam belas": 16, "ketujuh belas": 17, "kedelapan belas": 18, "kesembilan belas": 19, "kedua puluh": 20,
            "kedua puluh satu": 21, "kedua puluh dua": 22, "kedua puluh tiga": 23, "kedua puluh empat": 24, "kedua puluh lima": 25,
            "kedua puluh enam": 26, "kedua puluh tujuh": 27, "kedua puluh delapan": 28, "kedua puluh sembilan": 29, "ketiga puluh": 30,
            "ketiga puluh satu": 31, "ketiga puluh dua": 32, "ketiga puluh tiga": 33, "ketiga puluh empat": 34, "ketiga puluh lima": 35,
            "ketiga puluh enam": 36, "ketiga puluh tujuh": 37, "ketiga puluh delapan": 38, "ketiga puluh sembilan": 39, "keempat puluh": 40,
            "keempat puluh satu": 41, "keempat puluh dua": 42, "keempat puluh tiga": 43, "keempat puluh empat": 44, "keempat puluh lima": 45,
            "keempat puluh enam": 46, "keempat puluh tujuh": 47, "keempat puluh delapan": 48, "keempat puluh sembilan": 49, "kelima puluh": 50
        }
        self.REGULATION_ENCODING = {
            'type': {
                'UU': '01',
                'PERPPU': '02',
                'PP': '03',
                'PERPRES': '04',
                'PERMENKOMINFO': '05'
            },
            'section': {
                'document': '1',
                'considering': '2',
                'observing': '3',
                'definition': '4',
                'chapter': '5',
                'article': '6',
                'section': '7',
            }
        }
        self.REGEX_PATTERNS = {
            'document': {
                'metadata': r'^(\w+)_(\w+)_(\w+)'  # Jenis, tahun, dan nomor peraturan
            },
            'main': {
                'considering': r'(?<=## menimbang)([\S\s]*?)(?=## mengingat)',                                                 # Menimbang
                'observing': r'(?<=## mengingat)([\S\s]*?)(?=(?:dengan persetujuan bersama|## memperhatikan|## memutuskan))',  # Mengingat
                'amendment_to': r'^Perubahan',                                                                                 # Cek Peraturan Amandemen
                'chapter': r'(## BAB[\S\s]*?)(?=\s+(?:## BAB|agar setiap orang mengetahuinya|ditetapkan di))'                  # Daftar Bab
            },
            'chapter': {
                # For every chapters
                'about': r'## (BAB [^#]+)##',                                    # Nama Bab
                'part': r'(## Bagian [\S\s]*?)(?=\s+(?:## Bagian|$))',           # Daftar Bagian
                'paragraph': r'(## Paragraf [\S\s]*?)(?=\s+(?:## Paragraf|$))',  # Daftar Paragraf
                'article': r'(## Pasal \w+[\S\s]*?)(?=(?:##|$))'                 # Daftar Pasal
            },
            'part': {
                # For every parts
                'about': r'## (Bagian [^#]+)##',  # Nama Bagian
                'number': r'Bagian (\w+) -'       # Nomor Bagian
            },
            'paragraph': {
                # For every paragraphs
                'about': r'## (Paragraf [^#]+)##',  # Nama Paragraf
                'number': r'Paragraf (\w+) -'       # Nomor Paragraf

            },
            'article': {
                # For every articles
                'number': r'## Pasal (\d+\w*)',                                       # Nomor Pasal
                'text': r'## Pasal \w+\n*([\S\s]*)',                                  # Isi Pasal
                'check_definition': r'^dalam (?:undang-undang|peraturan)',            # Cek apakah Pasal 1 adalah definisi
                'definition': r'\(\d+[a-z]?\) (([A-Z][a-z]*(?:\s[A-Z][a-z]*)*) .*)',  # Daftar Definisi
                'reference_1': r'Pasal (\d+\w*)',                                     # Pasal Referensi Jenis 1
                'reference_2': r'Pasal (\d+\w*) sampai dengan Pasal (\d+\w*)'         # Pasal Referensi Jenis 2
            },
            'amendment_to': {
                'amendment_point_1': r'(## \d+\.[\S\s]*?)(?=\s+(?:## \d+\.|## Pasal II))',                      # Poin Amandemen Jenis 1
                'amendment_point_2': r'(?<=## Pasal I)([\s\S]*?)(?=## Pasal II)',                               # Poin Amandemen Jenis 2
                'chapter': r'(## BAB[\S\s]*?)(?=\s+(?:## BAB|agar setiap orang mengetahuinya|ditetapkan di))',  # Daftar Bab
                'part': r'(## Bagian [\S\s]*?)(?=\s+(?:## Bagian|$))',                                          # Daftar Bagian
                'paragraph': r'(## Paragraf [\S\s]*?)(?=\s+(?:## Paragraf|$))',                                 # Daftar Paragraf
                # 'amendment_articles': r'(## Pasal \d+[A-Z]?[\S\s]*?)(?=(?:##|$))',                            # Pasal Amandemen
            }
        }

    def parse_regulations_content(
            self,
            input_dir: str,
            json_input: str,
            json_output: str,
            verbose: bool = True
    ) -> list[dict]:

        # Initialize data
        article_dict = dict()
        durations = list()
        result = list()
        files = list()
        success = 0
        failed = 0

        # Get all Markdown files path and name
        for filename in os.listdir(input_dir):
            if filename.endswith(".md"):
                files.append((os.path.join(input_dir, filename), filename))

        # Iterate for every files
        for regulation_file in tqdm(iterable=files, desc='Parsing regulations content', disable=not verbose):
            start_time = time.time()
            filepath, filename = regulation_file

            try:
                with open(filepath, 'r', encoding='utf8') as file:
                    # Initialize data
                    text = file.read()
                    regulation_dict = dict()
                    definition_list = list()

                    # Get the file metadata
                    metadata = re.search(self.REGEX_PATTERNS['document']['metadata'], filename)
                    regulation_type = self.REGULATION_ENCODING['type'][metadata[1]]
                    regulation_year = metadata[2]
                    regulation_num = int(metadata[3])

                    # Create template ID
                    id_template = f'{regulation_year}{regulation_type}{str(regulation_num).zfill(3)}' \
                        + '{reg_section}{section_num}{extra_section_number}'

                    # Create regulation ID
                    regulation_id = id_template.format(
                        reg_section=self.REGULATION_ENCODING['section']['document'],
                        section_num='000',
                        extra_section_number='00'
                    )

                    # Get regulation data from regulation JSON
                    with open(json_input) as json_data:
                        for regulation_data in json.load(json_data):
                            if regulation_data['id'] == regulation_id:
                                regulation_dict = regulation_data

                    # Initialize the dictionary of parsing results
                    regulation_dict['content'] = dict()

                    # Get considering text (Menimbang)
                    regulation_dict['content']['considering'] = {
                        'id': id_template.format(
                            reg_section=self.REGULATION_ENCODING['section']['considering'],
                            section_num='000',
                            extra_section_number='00'
                        ),
                        'text': re.search(self.REGEX_PATTERNS['main']['considering'], text, re.IGNORECASE)[1].strip()
                    }

                    # Get observing text (Mengingat)
                    regulation_dict['content']['observing'] = {
                        'id': id_template.format(
                            reg_section=self.REGULATION_ENCODING['section']['observing'],
                            section_num='000',
                            extra_section_number='00'
                        ), 
                        'text': re.search(self.REGEX_PATTERNS['main']['observing'], text, re.IGNORECASE)[1].strip()
                    }

                    # Check for amendment regulation
                    is_amendment = re.search(self.REGEX_PATTERNS['main']['amendment_to'], regulation_dict['about'], re.IGNORECASE)
                    
                    if is_amendment:
                        regulation_dict, definition_list, article_dict = self.__parse_amendment_regulation(
                            text=text,
                            id_template=id_template,
                            regulation_dict=regulation_dict,
                            definition_list=definition_list,
                            article_dict=article_dict,
                            amend_regulations=regulation_dict['status']['amend']
                        )
                    else:
                        regulation_dict, definition_list, article_dict = self.__parse_base_regulation(
                            text=text,
                            id_template=id_template,
                            regulation_dict=regulation_dict,
                            definition_list=definition_list,
                            article_dict=article_dict
                        )
                    
                    result.append(regulation_dict)
                    success += 1
                
            except Exception as e:
                if verbose:
                    failed += 1
                    print(f'ERROR parsing content of {filename}')
                    print(e)

            durations.append(time.time() - start_time)

        if not json_output.endswith('.json'):
            json_output = f'{json_output}.json'

        with open(json_output, "w") as output_file: 
            json.dump(result, output_file, indent=4)
        
        if verbose:
            print('=' * 76)
            print(f'Input directory   : {input_dir}')
            print(f'Input JSON        : {json_input}')
            print(f'Output JSON       : {json_output}')
            print(f'Total regulations : {len(files)} regulations')
            print(f'Total success     : {success} regulations')
            print(f'Total failed      : {failed} regulations')
            print(f'Total articles    : {len(article_dict)} articles')
            print(f'Total time        : {round(sum(durations) * 1000, 3)} miliseconds')
            print(f'Average time/file : {round(sum(durations) * 1000 / success, 3)} miliseconds')
            print('=' * 76)

        return result
    

    def __parse_base_regulation(
            self,
            text: str,
            id_template: str,
            regulation_dict: dict,
            definition_list: list,
            article_dict: dict
    ) -> tuple[dict, list, dict]:

        regulation_dict['content']['articles'] = dict()

        # Get all chapters
        chapters = re.findall(self.REGEX_PATTERNS['main']['chapter'], text, re.IGNORECASE)

        # If the chapter exists
        if chapters:
            # Iterate for every chapters
            for chapter_num, chapter in enumerate(chapters):
                chapter_number = chapter_num + 1
                chapter_about = re.search(self.REGEX_PATTERNS['chapter']['about'], chapter, re.IGNORECASE)[1].strip().upper()
                chapter_about = re.sub(r'\n', ' - ', chapter_about, flags=re.IGNORECASE)

                # Get all parts
                parts = re.findall(self.REGEX_PATTERNS['chapter']['part'], chapter.strip() + '\n', re.IGNORECASE)

                # If the part exists
                if parts:
                    # Iterate for every parts
                    for part_num, part in enumerate(parts):
                        # Get part about/name
                        part_number = part_num + 1
                        part_about = re.search(self.REGEX_PATTERNS['part']['about'], part, re.IGNORECASE)[1].strip()
                        part_about = re.sub(r'\n', ' - ', part_about, flags=re.IGNORECASE)

                        # Get all paragraphs
                        paragraphs = re.findall(self.REGEX_PATTERNS['chapter']['paragraph'], part.strip() + '\n', re.IGNORECASE)
                        
                        # If the paragraph exists
                        if paragraphs:
                            # Iterate for every paragraphs
                            for paragraph_num, paragraph in enumerate(paragraphs):
                                # Get paragraph about/name
                                paragraph_number = paragraph_num + 1
                                paragraph_about = re.search(self.REGEX_PATTERNS['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
                                paragraph_about = re.sub(r'\n', ' - ', paragraph_about, flags=re.IGNORECASE)
                                
                                regulation_dict, definition_list, article_dict = self.__parse_articles(
                                    text=paragraph,
                                    chapter_number=str(chapter_number),
                                    chapter_about=chapter_about,
                                    part_number=str(part_number),
                                    part_about=part_about,
                                    paragraph_number=str(paragraph_number),
                                    paragraph_about=paragraph_about,
                                    id_template=id_template,
                                    regulation_dict=regulation_dict,
                                    definition_list=definition_list,
                                    article_dict=article_dict
                                )

                        else:
                            regulation_dict, definition_list, article_dict = self.__parse_articles(
                                text=part,
                                chapter_number=str(chapter_number),
                                chapter_about=chapter_about,
                                part_number=str(part_number),
                                part_about=part_about,
                                paragraph_number='',
                                paragraph_about='',
                                id_template=id_template,
                                regulation_dict=regulation_dict,
                                definition_list=definition_list,
                                article_dict=article_dict
                            )

                else:
                    regulation_dict, definition_list, article_dict = self.__parse_articles(
                        text=chapter,
                        chapter_number=str(chapter_number),
                        chapter_about=chapter_about,
                        part_number='',
                        part_about='',
                        paragraph_number='',
                        paragraph_about='',
                        id_template=id_template,
                        regulation_dict=regulation_dict,
                        definition_list=definition_list,
                        article_dict=article_dict
                    )
        else:
            regulation_dict, definition_list, article_dict = self.__parse_articles(
                text=text,
                chapter_number='',
                chapter_about='',
                part_number='',
                part_about='',
                paragraph_number='',
                paragraph_about='',
                id_template=id_template,
                regulation_dict=regulation_dict,
                definition_list=definition_list,
                article_dict=article_dict
            )


        return regulation_dict, definition_list, article_dict


    def __parse_amendment_regulation(
            self,
            text: str,
            id_template: str,
            regulation_dict: dict,
            definition_list: list,
            article_dict: dict,
            amend_regulations: list
    ) -> tuple[dict, list, dict]:
        
        regulation_dict['content']['articles'] = dict()
        
        # Get all amendment points
        amendment_points = re.findall(self.REGEX_PATTERNS['amendment_to']['amendment_point_1'], text, re.IGNORECASE)

        if not amendment_points:
            amendment_points = re.search(self.REGEX_PATTERNS['amendment_to']['amendment_point_2'], text, re.IGNORECASE)[1].strip()
            first_sentence = re.search(r'^.*', amendment_points, re.IGNORECASE)[0].strip()  # Dapatkan kalimat pertama
            amendment_points = [amendment_points.replace(first_sentence, '').strip()]		# Hapus kalimat pertama
        
        # Iterate for every points
        for point in amendment_points:
            

            # Get all parts
            parts = re.findall(self.REGEX_PATTERNS['amendment_to']['part'], point.strip() + '\n', re.IGNORECASE)

            # If the part exists
            if parts:
                # Iterate for every parts
                for part in parts:
                    # Get part about/name
                    part_about = re.search(self.REGEX_PATTERNS['part']['about'], part, re.IGNORECASE)[1].strip()
                    part_about = re.sub(r'\n', ' - ', part_about, flags=re.IGNORECASE)
                    part_number = re.search(self.REGEX_PATTERNS['part']['number'], part_about, re.IGNORECASE)[1].strip().lower()
                    part_number = self.WORD_TO_NUMBER.get(part_number, 0)

                    # Get all paragraphs
                    paragraphs = re.findall(self.REGEX_PATTERNS['chapter']['paragraph'], part.strip() + '\n', re.IGNORECASE)
                    
                    # If the paragraph exists
                    if paragraphs:
                        # Iterate for every paragraphs
                        for paragraph in paragraphs:
                            # Get paragraph about/name
                            paragraph_about = re.search(self.REGEX_PATTERNS['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
                            paragraph_about = re.sub(r'\n', ' - ', paragraph_about, flags=re.IGNORECASE)
                            paragraph_number = re.search(self.REGEX_PATTERNS['paragraph']['number'], paragraph_about, re.IGNORECASE)[1].strip()
                            
                            regulation_dict, definition_list, article_dict = self.__parse_articles(
                                text=paragraph,
                                chapter_number='',
                                chapter_about='',
                                part_number=str(part_number),
                                part_about=part_about,
                                paragraph_number=str(paragraph_number),
                                paragraph_about=paragraph_about,
                                id_template=id_template,
                                regulation_dict=regulation_dict,
                                definition_list=definition_list,
                                article_dict=article_dict,
                                amendment=True,
                                amend_regulations=amend_regulations
                            )

                    else:
                        regulation_dict, definition_list, article_dict = self.__parse_articles(
                            text=part,
                            chapter_number='',
                            chapter_about='',
                            part_number=str(part_number),
                            part_about=part_about,
                            paragraph_number='',
                            paragraph_about='',
                            id_template=id_template,
                            regulation_dict=regulation_dict,
                            definition_list=definition_list,
                            article_dict=article_dict,
                            amendment=True,
                            amend_regulations=amend_regulations
                        )

            else:
                # Get all paragraphs
                paragraphs = re.findall(self.REGEX_PATTERNS['amendment_to']['paragraph'], point.strip() + '\n', re.IGNORECASE)
                
                # If the paragraph exists
                if paragraphs:
                    # Iterate for every paragraphs
                    for paragraph_num, paragraph in enumerate(paragraphs):
                        # Get paragraph about/name
                        paragraph_number = paragraph_num + 1
                        paragraph_about = re.search(self.REGEX_PATTERNS['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
                        paragraph_about = re.sub(r'\n', ' - ', paragraph_about, flags=re.IGNORECASE)
                        paragraph_number = re.search(self.REGEX_PATTERNS['paragraph']['number'], paragraph_about, re.IGNORECASE)[1].strip()

                        regulation_dict, definition_list, article_dict = self.__parse_articles(
                            text=paragraph,
                            chapter_number='',
                            chapter_about='',
                            part_number='',
                            part_about='',
                            paragraph_number=str(paragraph_number),
                            paragraph_about=paragraph_about,
                            id_template=id_template,
                            regulation_dict=regulation_dict,
                            definition_list=definition_list,
                            article_dict=article_dict,
                            amendment=True,
                            amend_regulations=amend_regulations
                        )

                else:
                    regulation_dict, definition_list, article_dict = self.__parse_articles(
                        text=point,
                        chapter_number='',
                        chapter_about='',
                        part_number='',
                        part_about='',
                        paragraph_number='',
                        paragraph_about='',
                        id_template=id_template,
                        regulation_dict=regulation_dict,
                        definition_list=definition_list,
                        article_dict=article_dict,
                        amendment=True,
                        amend_regulations=amend_regulations
                    )

        return regulation_dict, definition_list, article_dict


    def __parse_articles(
            self,
            text: str,
            chapter_number: str,
            chapter_about: str,
            part_number: str,
            part_about: str,
            paragraph_number: str,
            paragraph_about: str,
            id_template: str,
            regulation_dict: dict,
            definition_list: list,
            article_dict: dict,
            amendment: bool = False,
            amend_regulations: list = list() 
    ) -> tuple[dict, list]:
        
        # Get all articles
        articles = re.findall(self.REGEX_PATTERNS['chapter']['article'], text, re.IGNORECASE)

        # Iterate for every articles
        for article in articles:
            article_number = re.search(self.REGEX_PATTERNS['article']['number'], article, re.IGNORECASE)[1]
            article_text = re.search(self.REGEX_PATTERNS['article']['text'], article, re.IGNORECASE)[1].strip()
            article_text = re.sub(r'\n+', '\n', article_text)
            
            # Get definition (Article/Pasal 1)
            if article_number == '1':
                if re.search(self.REGEX_PATTERNS['article']['check_definition'], article_text, re.IGNORECASE):
                    definitions = re.findall(self.REGEX_PATTERNS['article']['definition'], article_text)

                    for index, definition_data in enumerate(definitions):
                        definition, name = definition_data
                        definition_list.append({
                            'id': id_template.format(
                                reg_section=self.REGULATION_ENCODING['section']['definition'],
                                section_num=str(index + 1).zfill(3),
                                extra_section_number='00'
                            ),
                            'name': name.strip(),
                            'definition': definition.strip()
                        })

            # Store definition list
            regulation_dict['content']['definitions'] = definition_list

            # Define article ID
            article_id = ''

            if amendment:
                # Create article ID
                article_alphabet = re.search(r'\d+([A-Z]+)', str(article_number), re.IGNORECASE)

                if article_alphabet:
                    article_id = id_template.format(
                        reg_section=self.REGULATION_ENCODING['section']['article'],
                        section_num=re.search(r'\d+', str(article_number), re.IGNORECASE)[0].zfill(3),
                        extra_section_number=self.ALPHABET_TO_NUMERIC[article_alphabet[1]]
                    )
                else:
                    article_id = id_template.format(
                        reg_section=self.REGULATION_ENCODING['section']['article'],
                        section_num=str(article_number).zfill(3),
                        extra_section_number='00'
                    )

                # Store article data
                regulation_dict['content']['articles'][article_number] = {
                    'id': article_id,
                    'chapter_number': chapter_number,
                    'chapter_about': chapter_about,
                    'part_number': part_number,
                    'part_about': part_about,
                    'paragraph_number': paragraph_number,
                    'paragraph_about': paragraph_about,
                    'article_number': article_number,
                    'text': article_text
                }

                # Store '' as previous article ID (for consistency with base regulation / not amandment)
                regulation_dict['content']['articles'][article_number]['previous_article'] = ''

                # Store empty list as 'references' (for consistency with base regulation / not amandment)
                regulation_dict['content']['articles'][article_number]['references'] = list()

                # Mendefinisikan daftar pasal yang diamandemen
                regulation_dict['content']['articles'][article_number]['amend'] = list()
                
                # Ambil daftar peraturan amend
                for regulation_id in amend_regulations:
                    # Buat ulang (tebak) ID Pasal di peraturan sebelum revisi
                    other_article_id = regulation_id[:-6] + '6' + article_id[-5:]

                    # Jika ID tersebut ada di article_dict maka ID nya pasti ada, terus simpan 
                    if other_article_id in article_dict.keys():

                        # BINGUNG ANTARA MAU PAKAI ATAU TIDAK: Chapter, Part, Paragraph
                        # Article ID FIX PAKAI

                        # Chapter
                        regulation_dict['content']['articles'][article_number]['chapter_number'] = \
                            chapter_number if chapter_number else article_dict[other_article_id]['chapter_number']
                        regulation_dict['content']['articles'][article_number]['chapter_about'] = \
                            chapter_about if chapter_about else article_dict[other_article_id]['chapter_about']
                        
                        # Part
                        regulation_dict['content']['articles'][article_number]['part_number'] = \
                            part_number if part_number else article_dict[other_article_id]['part_number']
                        regulation_dict['content']['articles'][article_number]['part_about'] = \
                            part_about if part_about else article_dict[other_article_id]['part_about']
                        
                        # Paragraph
                        regulation_dict['content']['articles'][article_number]['paragraph_number'] = \
                            paragraph_number if paragraph_number else article_dict[other_article_id]['paragraph_number']
                        regulation_dict['content']['articles'][article_number]['paragraph_about'] = \
                            paragraph_about if paragraph_about else article_dict[other_article_id]['paragraph_about']
                        
                        # Article ID
                        regulation_dict['content']['articles'][article_number]['amend'].append(other_article_id)
                
                # Store article to article_dict
                article_dict[article_id] = regulation_dict['content']['articles'][article_number]

            else:
                # Create article ID
                article_id = id_template.format(
                    reg_section=self.REGULATION_ENCODING['section']['article'],
                    section_num=str(article_number).zfill(3),
                    extra_section_number='00'
                )

                # Store article
                regulation_dict['content']['articles'][article_number] = {
                    'id': article_id,
                    'chapter_number': chapter_number,
                    'chapter_about': chapter_about,
                    'part_number': part_number,
                    'part_about': part_about,
                    'paragraph_number': paragraph_number,
                    'paragraph_about': paragraph_about,
                    'article_number': article_number,
                    'text': article_text
                }

                # Get previous article ID
                if article_number != '1':
                    regulation_dict['content']['articles'][article_number]['previous_article'] = \
                        id_template.format(
                            reg_section=self.REGULATION_ENCODING['section']['article'],
                            section_num=str(int(article_number) - 1).zfill(3),
                            extra_section_number='00'
                        )
                else:
                    regulation_dict['content']['articles'][article_number]['previous_article'] = ''
                
                
                # Get article reference to other article
                regulation_dict['content']['articles'][article_number]['references'] = list()
                reference_type_1 = list(set(re.findall(self.REGEX_PATTERNS['article']['reference_1'], article_text, re.IGNORECASE)))
                reference_type_2 = list(set(re.findall(self.REGEX_PATTERNS['article']['reference_2'], article_text, re.IGNORECASE)))

                article_references = self.__merge_article_references(reference_type_1, reference_type_2)

                for article_reference_num in article_references:
                    regulation_dict['content']['articles'][article_number]['references'].append(
                        id_template.format(
                            reg_section=self.REGULATION_ENCODING['section']['article'],
                            section_num=str(article_reference_num).zfill(3),
                            extra_section_number='00'
                        )
                    )

                # Store empty list as 'amend' (for consistency with amandment regulation)
                regulation_dict['content']['articles'][article_number]['amend'] = list()
                        
                # Store article to article_dict
                article_dict[article_id] = regulation_dict['content']['articles'][article_number]

        return regulation_dict, definition_list, article_dict


    def __merge_article_references(self, list1: list, list2: list) -> list:
        set1 = set(int(x) for x in list1)
        set2 = set()

        for start, end in list2:
            set2.update(range(int(start), int(end) + 1))

        return sorted(set1.union(set2))

In [ ]:
input_dir = os.path.join('data', 'markdown', 'fix', 'temp')
json_input = os.path.join('data', 'regulation_data_modified.json')
json_output = os.path.join('data', 'regulation_data_final.json')

parser = RegulationParser()
regulation_data = parser.parse_regulations_content(
    input_dir=input_dir,
    json_input=json_input,
    json_output=json_output,
    verbose=True
)

In [ ]:
# # MODEL 1: STRUKTUR PERATURAN DASAR DAN AMANDEMEN BERBEDA

# class RegulationParser:

#     def __init__(self):
#         self.ALPHABET_TO_NUMERIC = {chr(i): f'{i - 64:02}' for i in range(65, 91)}  # A-Z -> 01-26
#         self.REGULATION_ENCODING = {
#             'type': {
#                 'UU': '01',
#                 'PERPPU': '02',
#                 'PP': '03',
#                 'PERPRES': '04',
#                 'PERMENKOMINFO': '05'
#             },
#             'section': {
#                 'document': '1',
#                 'considering': '2',
#                 'observing': '3',
#                 'definition': '4',
#                 'chapter': '5',
#                 'article': '6',
#                 'section': '7',
#             }
#         }
#         self.REGEX_PATTERNS = {
#             'document': {
#                 'metadata': r'^(\w+)_(\w+)_(\w+)'  # Jenis, tahun, dan nomor peraturan
#             },
#             'main': {
#                 'considering': r'(?<=## menimbang)([\S\s]*?)(?=## mengingat)',                                                 # Menimbang
#                 'observing': r'(?<=## mengingat)([\S\s]*?)(?=(?:dengan persetujuan bersama|## memperhatikan|## memutuskan))',  # Mengingat
#                 'amendment_to': r'^Perubahan',                                                                                 # Cek Peraturan Amandemen
#                 'chapter': r'(## BAB[\S\s]*?)(?=\s+(?:## BAB|agar setiap orang mengetahuinya|ditetapkan di))'                  # Daftar Bab
#             },
#             'chapter': {
#                 # For every chapters
#                 'about': r'## (BAB [\w\s\-\/\,]+)##',                            # Nama Bab, ganti [\-\/\,] dengan semua tanda baca kecuali #
#                 'part': r'(## Bagian [\S\s]*?)(?=\s+(?:## Bagian|$))',           # Daftar Bagian
#                 'paragraph': r'(## Paragraf [\S\s]*?)(?=\s+(?:## Paragraf|$))',  # Daftar Paragraf
#                 'article': r'(## Pasal \w+[\S\s]*?)(?=(?:##|$))'                 # Daftar Pasal
#             },
#             'part': {
#                 # For every parts
#                 'about': r'## (Bagian [\w\s\-\/\,]+)##'  # Nama Bagian
#             },
#             'paragraph': {
#                 # For every paragraphs
#                 'about': r'## (Paragraf [\w\s\-\/\,]+)##'  # Nama Paragraf
#             },
#             'article': {
#                 # For every articles
#                 'number': r'## Pasal (\d+\w*)',                                       # Nomor Pasal
#                 'text': r'## Pasal \w+\n*([\S\s]*)',                                  # Isi Pasal
#                 'definition': r'\(\d+[a-z]?\) (([A-Z][a-z]*(?:\s[A-Z][a-z]*)*) .*)',  # Daftar Definisi
#                 'reference_1': r'Pasal (\d+\w*)',                                     # Pasal Referensi Jenis 1
#                 'reference_2': r'Pasal (\d+\w*) sampai dengan Pasal (\d+\w*)'         # Pasal Referensi Jenis 2
#             },
#             'amendment_to': {
#                 'amendment_point_1': r'(## \d+\.[\S\s]*?)(?=\s+(?:## \d+\.|## Pasal II))',  # Poin Amandemen Jenis 1
#                 'amendment_point_2': r'(?<=## Pasal I)([\s\S]*?)(?=## Pasal II)',           # Poin Amandemen Jenis 2
#                 'part': r'(## Bagian [\S\s]*?)(?=\s+(?:## Bagian|$))',                      # Daftar Bagian
#                 'paragraph': r'(## Paragraf [\S\s]*?)(?=\s+(?:## Paragraf|$))',             # Daftar Paragraf
#                 # 'amendment_articles': r'(## Pasal \d+[A-Z]?[\S\s]*?)(?=(?:##|$))',          # Pasal Amandemen
#             }
#         }

#     def parse_regulations_content(
#             self,
#             input_dir: str,
#             json_input: str,
#             json_output: str,
#             verbose: bool = True
#     ) -> list[dict]:

#         # Initialize data
#         article_dict = dict()
#         durations = list()
#         result = list()
#         files = list()
#         success = 0
#         failed = 0

#         # Get all Markdown files path and name
#         for filename in os.listdir(input_dir):
#             if filename.endswith(".md"):
#                 files.append((os.path.join(input_dir, filename), filename))

#         # Iterate for every files
#         for regulation_file in tqdm(iterable=files, desc='Parsing regulations content', disable=not verbose):
#             start_time = time.time()
#             filepath, filename = regulation_file

#             try:
#                 with open(filepath, 'r', encoding='utf8') as file:
#                     # Initialize data
#                     text = file.read()
#                     regulation_dict = dict()
#                     definition_list = list()

#                     # Get the file metadata
#                     metadata = re.search(self.REGEX_PATTERNS['document']['metadata'], filename)
#                     regulation_type = self.REGULATION_ENCODING['type'][metadata[1]]
#                     regulation_year = metadata[2]
#                     regulation_num = int(metadata[3])

#                     # Create template ID
#                     id_template = f'{regulation_year}{regulation_type}{str(regulation_num).zfill(3)}' \
#                         + '{reg_section}{section_num}{extra_section_number}'

#                     # Create regulation ID
#                     regulation_id = id_template.format(
#                         reg_section=self.REGULATION_ENCODING['section']['document'],
#                         section_num='000',
#                         extra_section_number='00'
#                     )

#                     # Get regulation data from regulation JSON
#                     with open(json_input) as json_data:
#                         for regulation_data in json.load(json_data):
#                             if regulation_data['id'] == regulation_id:
#                                 regulation_dict = regulation_data

#                     # Initialize the dictionary of parsing results
#                     regulation_dict['content'] = dict()

#                     # Get considering text (Menimbang)
#                     regulation_dict['content']['considering'] = {
#                         'id': id_template.format(
#                             reg_section=self.REGULATION_ENCODING['section']['considering'],
#                             section_num='000',
#                             extra_section_number='00'
#                         ),
#                         'text': re.search(self.REGEX_PATTERNS['main']['considering'], text, re.IGNORECASE)[1].strip()
#                     }

#                     # Get observing text (Mengingat)
#                     regulation_dict['content']['observing'] = {
#                         'id': id_template.format(
#                             reg_section=self.REGULATION_ENCODING['section']['observing'],
#                             section_num='000',
#                             extra_section_number='00'
#                         ), 
#                         'text': re.search(self.REGEX_PATTERNS['main']['observing'], text, re.IGNORECASE)[1].strip()
#                     }

#                     # Check for amendment regulation
#                     is_amendment = re.search(self.REGEX_PATTERNS['main']['amendment_to'], regulation_dict['about'], re.IGNORECASE)
                    
#                     if is_amendment:
#                         regulation_dict, definition_list, article_dict = self.__parse_amendment_regulation(
#                             text=text,
#                             id_template=id_template,
#                             regulation_dict=regulation_dict,
#                             definition_list=definition_list,
#                             article_dict=article_dict,
#                             amend_regulations=regulation_dict['status']['amend']
#                         )
#                     else:
#                         regulation_dict, definition_list, article_dict = self.__parse_base_regulation(
#                             text=text,
#                             id_template=id_template,
#                             regulation_dict=regulation_dict,
#                             definition_list=definition_list,
#                             article_dict=article_dict
#                         )
                    
#                     result.append(regulation_dict)
#                     success += 1
                
#             except Exception as e:
#                 if verbose:
#                     failed += 1
#                     print(f'ERROR parsing content of {filename}')
#                     print(e)

#             durations.append(time.time() - start_time)

#         if not json_output.endswith('.json'):
#             json_output = f'{json_output}.json'

#         with open(json_output, "w") as output_file: 
#             json.dump(result, output_file, indent=4)
        
#         if verbose:
#             print('=' * 76)
#             print(f'Input directory   : {input_dir}')
#             print(f'Input JSON        : {json_input}')
#             print(f'Output JSON       : {json_output}')
#             print(f'Total regulations : {len(files)} regulations')
#             print(f'Total success     : {success} regulations')
#             print(f'Total failed      : {failed} regulations')
#             print(f'Total articles    : {len(article_dict)} articles')
#             print(f'Total time        : {round(sum(durations) * 1000, 3)} miliseconds')
#             print(f'Average time/file : {round(sum(durations) * 1000 / success, 3)} miliseconds')
#             print('=' * 76)

#         return result
    

#     def __parse_base_regulation(
#             self,
#             text: str,
#             id_template: str,
#             regulation_dict: dict,
#             definition_list: list,
#             article_dict: dict
#     ) -> tuple[dict, list, dict]:

#         regulation_dict['content']['chapters'] = dict()

#         # Get all chapters
#         chapters = re.findall(self.REGEX_PATTERNS['main']['chapter'], text, re.IGNORECASE)

#         # Iterate for every chapters
#         for num, chapter in enumerate(chapters):
#             chapter_num = num + 1
#             chapter_about = re.search(self.REGEX_PATTERNS['chapter']['about'], chapter, re.IGNORECASE)[1].strip().upper()
#             chapter_about = re.sub(r'\n', ': ', chapter_about, flags=re.IGNORECASE)

#             regulation_dict['content']['chapters'][chapter_num] = {
#                 'id': id_template.format(
#                     reg_section=self.REGULATION_ENCODING['section']['chapter'],
#                     section_num=str(chapter_num).zfill(3),
#                     extra_section_number='00'
#                 ),
#                 'about': chapter_about
#             }
        
#             regulation_dict['content']['chapters'][chapter_num]['articles'] = dict()

#             # Get all parts
#             parts = re.findall(self.REGEX_PATTERNS['chapter']['part'], chapter.strip() + '\n', re.IGNORECASE)

#             # If the part exists
#             if parts:
#                 # Iterate for every parts
#                 for part in parts:
#                     # Get part about/name
#                     part_about = re.search(self.REGEX_PATTERNS['part']['about'], part, re.IGNORECASE)[1].strip()
#                     part_about = re.sub(r'\n', ': ', part_about, flags=re.IGNORECASE)

#                     # Get all paragraphs
#                     paragraphs = re.findall(self.REGEX_PATTERNS['chapter']['paragraph'], part.strip() + '\n', re.IGNORECASE)
                    
#                     # If the paragraph exists
#                     if paragraphs:
#                         # Iterate for every paragraphs
#                         for paragraph in paragraphs:
#                             # Get paragraph about/name
#                             paragraph_about = re.search(self.REGEX_PATTERNS['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
#                             paragraph_about = re.sub(r'\n', ': ', paragraph_about, flags=re.IGNORECASE)
                            
#                             regulation_dict, definition_list, article_dict = self.__parse_articles(
#                                 text=paragraph,
#                                 chapter_num=chapter_num,
#                                 part_about=part_about,
#                                 paragraph_about=paragraph_about,
#                                 id_template=id_template,
#                                 regulation_dict=regulation_dict,
#                                 definition_list=definition_list,
#                                 article_dict=article_dict
#                             )

#                     else:
#                         regulation_dict, definition_list, article_dict = self.__parse_articles(
#                             text=part,
#                             chapter_num=chapter_num,
#                             part_about=part_about,
#                             paragraph_about='',
#                             id_template=id_template,
#                             regulation_dict=regulation_dict,
#                             definition_list=definition_list,
#                             article_dict=article_dict
#                         )

#             else:
#                 regulation_dict, definition_list, article_dict = self.__parse_articles(
#                     text=chapter,
#                     chapter_num=chapter_num,
#                     part_about='',
#                     paragraph_about='',
#                     id_template=id_template,
#                     regulation_dict=regulation_dict,
#                     definition_list=definition_list,
#                     article_dict=article_dict
#                 )

#         return regulation_dict, definition_list, article_dict


#     def __parse_amendment_regulation(
#             self,
#             text: str,
#             id_template: str,
#             regulation_dict: dict,
#             definition_list: list,
#             article_dict: dict,
#             amend_regulations: list
#     ) -> tuple[dict, list, dict]:
        
#         # MUNGKIN INI BUAT MASALAH? COBA DULU
#         regulation_dict['content']['articles'] = dict()
        
#         # Get all amendment points
#         amendment_points = re.findall(self.REGEX_PATTERNS['amendment_to']['amendment_point_1'], text, re.IGNORECASE)

#         if not amendment_points:
#             amendment_points = re.search(self.REGEX_PATTERNS['amendment_to']['amendment_point_2'], text, re.IGNORECASE)[1].strip()
#             first_sentence = re.search(r'^.*', amendment_points, re.IGNORECASE)[0].strip()  # Dapatkan kalimat pertama
#             amendment_points = [amendment_points.replace(first_sentence, '').strip()]		# Hapus kalimat pertama
        
#         # Iterate for every points
#         for point in amendment_points:
#             # Get all parts
#             parts = re.findall(self.REGEX_PATTERNS['amendment_to']['part'], point.strip() + '\n', re.IGNORECASE)

#             # If the part exists
#             if parts:
#                 # Iterate for every parts
#                 for part in parts:
#                     # Get part about/name
#                     part_about = re.search(self.REGEX_PATTERNS['part']['about'], part, re.IGNORECASE)[1].strip()
#                     part_about = re.sub(r'\n', ': ', part_about, flags=re.IGNORECASE)

#                     # Get all paragraphs
#                     paragraphs = re.findall(self.REGEX_PATTERNS['chapter']['paragraph'], part.strip() + '\n', re.IGNORECASE)
                    
#                     # If the paragraph exists
#                     if paragraphs:
#                         # Iterate for every paragraphs
#                         for paragraph in paragraphs:
#                             # Get paragraph about/name
#                             paragraph_about = re.search(self.REGEX_PATTERNS['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
#                             paragraph_about = re.sub(r'\n', ': ', paragraph_about, flags=re.IGNORECASE)
                            
#                             regulation_dict, definition_list, article_dict = self.__parse_articles(
#                                 text=paragraph,
#                                 chapter_num=0,
#                                 part_about=part_about,
#                                 paragraph_about=paragraph_about,
#                                 id_template=id_template,
#                                 regulation_dict=regulation_dict,
#                                 definition_list=definition_list,
#                                 article_dict=article_dict,
#                                 amendment=True,
#                                 amend_regulations=amend_regulations
#                             )

#                     else:
#                         regulation_dict, definition_list, article_dict = self.__parse_articles(
#                             text=part,
#                             chapter_num=0,
#                             part_about=part_about,
#                             paragraph_about='',
#                             id_template=id_template,
#                             regulation_dict=regulation_dict,
#                             definition_list=definition_list,
#                             article_dict=article_dict,
#                             amendment=True,
#                             amend_regulations=amend_regulations
#                         )

#             else:
#                 # Get all paragraphs
#                 paragraphs = re.findall(self.REGEX_PATTERNS['amendment_to']['paragraph'], point.strip() + '\n', re.IGNORECASE)
                
#                 # If the paragraph exists
#                 if paragraphs:
#                     # Iterate for every paragraphs
#                     for paragraph in paragraphs:
#                         # Get paragraph about/name
#                         paragraph_about = re.search(self.REGEX_PATTERNS['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
#                         paragraph_about = re.sub(r'\n', ': ', paragraph_about, flags=re.IGNORECASE)

#                         regulation_dict, definition_list, article_dict = self.__parse_articles(
#                             text=paragraph,
#                             chapter_num=0,
#                             part_about='',
#                             paragraph_about=paragraph_about,
#                             id_template=id_template,
#                             regulation_dict=regulation_dict,
#                             definition_list=definition_list,
#                             article_dict=article_dict,
#                             amendment=True,
#                             amend_regulations=amend_regulations
#                         )

#                 else:
#                     regulation_dict, definition_list, article_dict = self.__parse_articles(
#                         text=point,
#                         chapter_num=0,
#                         part_about='',
#                         paragraph_about='',
#                         id_template=id_template,
#                         regulation_dict=regulation_dict,
#                         definition_list=definition_list,
#                         article_dict=article_dict,
#                         amendment=True,
#                         amend_regulations=amend_regulations
#                     )

#         return regulation_dict, definition_list, article_dict


#     def __parse_articles(
#             self,
#             text: str,
#             chapter_num: int,
#             part_about: str,
#             paragraph_about: str,
#             id_template: str,
#             regulation_dict: dict,
#             definition_list: list,
#             article_dict: dict,
#             amendment: bool = False,
#             amend_regulations: list = list() 
#     ) -> tuple[dict, list]:
        
#         # Get all articles
#         articles = re.findall(self.REGEX_PATTERNS['chapter']['article'], text, re.IGNORECASE)

#         # Iterate for every articles
#         for article in articles:
#             article_num = re.search(self.REGEX_PATTERNS['article']['number'], article, re.IGNORECASE)[1]
#             article_text = re.search(self.REGEX_PATTERNS['article']['text'], article, re.IGNORECASE)[1].strip()
#             article_text = re.sub(r'\n+', '\n', article_text)
            
#             # Get definition (Article/Pasal 1)
#             if article_num == '1':
#                 definitions = re.findall(self.REGEX_PATTERNS['article']['definition'], article_text)
#                 for index, definition_data in enumerate(definitions):
#                     definition, name = definition_data
#                     definition_list.append({
#                         'id': id_template.format(
#                             reg_section=self.REGULATION_ENCODING['section']['definition'],
#                             section_num=str(index + 1).zfill(3),
#                             extra_section_number='00'
#                         ),
#                         'name': name.strip(),
#                         'definition': definition.strip()
#                     })
#                 regulation_dict['content']['definitions'] = definition_list

#             # Define article ID
#             article_id = ''

#             if amendment:
#                 # Create article ID
#                 article_alphabet = re.search(r'\d+([A-Z]+)', str(article_num), re.IGNORECASE)

#                 if article_alphabet:
#                     article_id = id_template.format(
#                         reg_section=self.REGULATION_ENCODING['section']['article'],
#                         section_num=re.search(r'\d+', str(article_num), re.IGNORECASE)[0].zfill(3),
#                         extra_section_number=self.ALPHABET_TO_NUMERIC[article_alphabet[1]]
#                     )
#                 else:
#                     article_id = id_template.format(
#                         reg_section=self.REGULATION_ENCODING['section']['article'],
#                         section_num=str(article_num).zfill(3),
#                         extra_section_number='00'
#                     )

#                 # Store article data
#                 regulation_dict['content']['articles'][article_num] = {
#                     'id': article_id,
#                     'part': part_about,
#                     'paragraph': paragraph_about,
#                     'text': article_text
#                 }

#                 # Store article to article_dict
#                 article_dict[article_id] = regulation_dict['content']['articles'][article_num]

#                 # Mendefinisikan daftar pasasl yang diamandemen
#                 regulation_dict['content']['articles'][article_num]['amend'] = list()
                
#                 # Ambil daftar peraturan amend
#                 for regulation_id in amend_regulations:
#                     # Buat ulang (tebak) ID Pasal di peraturan sebelum revisi
#                     other_article_id = regulation_id[:-6] + '6' + article_id[-5:]

#                     # Jika ID tersebut ada di article_dict maka ID nya pasti ada, terus simpan 
#                     if other_article_id in article_dict.keys():
#                         regulation_dict['content']['articles'][article_num]['part'] = \
#                             part_about if part_about else article_dict[other_article_id]['part']
#                         regulation_dict['content']['articles'][article_num]['paragraph'] = \
#                             paragraph_about if paragraph_about else article_dict[other_article_id]['paragraph']
#                         regulation_dict['content']['articles'][article_num]['amend'].append(other_article_id)

#             else:
#                 # Create article ID
#                 article_id = id_template.format(
#                     reg_section=self.REGULATION_ENCODING['section']['article'],
#                     section_num=str(article_num).zfill(3),
#                     extra_section_number='00'
#                 )

#                 # Store article
#                 regulation_dict['content']['chapters'][chapter_num]['articles'][article_num] = {
#                     'id': article_id,
#                     'part': part_about,
#                     'paragraph': paragraph_about,
#                     'text': article_text
#                 }

#                 # Store article to article_dict
#                 article_dict[article_id] = regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]

#                 # Get previous article ID
#                 if article_num != '1':
#                     regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['previous_article'] = \
#                         id_template.format(
#                             reg_section=self.REGULATION_ENCODING['section']['article'],
#                             section_num=str(int(article_num) - 1).zfill(3),
#                             extra_section_number='00'
#                         )
#                 else:
#                     regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['previous_article'] = ''
                
#                 # Get article reference to other article
#                 regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['references'] = list()
#                 reference_type_1 = list(set(re.findall(self.REGEX_PATTERNS['article']['reference_1'], article_text, re.IGNORECASE)))
#                 reference_type_2 = list(set(re.findall(self.REGEX_PATTERNS['article']['reference_2'], article_text, re.IGNORECASE)))

#                 article_references = self.__merge_article_references(reference_type_1, reference_type_2)

#                 for article_reference_num in article_references:
#                     regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['references'].append(
#                         id_template.format(
#                             reg_section=self.REGULATION_ENCODING['section']['article'],
#                             section_num=str(article_reference_num).zfill(3),
#                             extra_section_number='00'
#                         )
#                     )
                        
#         return regulation_dict, definition_list, article_dict


#     def __merge_article_references(self, list1: list, list2: list) -> list:
#         set1 = set(int(x) for x in list1)
#         set2 = set()

#         for start, end in list2:
#             set2.update(range(int(start), int(end) + 1))

#         return sorted(set1.union(set2))

In [ ]:
# input_dir = os.path.join('data', 'markdown', 'fix', 'temp')
# json_input = os.path.join('data', 'regulation_data_modified.json')
# json_output = os.path.join('data', 'regulation_data_final.json')

# parser = RegulationParser()
# regulation_data = parser.parse_regulations_content(
#     input_dir=input_dir,
#     json_input=json_input,
#     json_output=json_output,
#     verbose=True
# )

In [ ]:
# mapping_alphabet_to_numeric = {chr(i): f'{i - 64:02}' for i in range(65, 91)}  # A-Z -> 01-26

# encode = {
#     'type': {
#         'UU': '01',
#         'PERPPU': '02',
#         'PP': '03',
#         'PERPRES': '04',
#         'PERMENKOMINFO': '05'
#     },
#     'section': {
#         'document': '1',
#         'considering': '2',
#         'observing': '3',
#         'definition': '4',
#         'chapter': '5',
#         'article': '6',
#         'section': '7',
#     }
# }

# patterns = {
#     'document': {
#         'metadata': r'^(\w+)_(\w+)_(\w+)'    # Jenis, tahun, dan nomor peraturan
#     },
#     'main': {
#         'considering': r'(?<=## menimbang)([\S\s]*?)(?=## mengingat)',                                                 # Menimbang
#         'observing': r'(?<=## mengingat)([\S\s]*?)(?=(?:dengan persetujuan bersama|## memperhatikan|## memutuskan))',  # Mengingat
#         'amendment_to': r'^Perubahan',                                                                                 # Cek Peraturan Revisi
#         'chapter': r'(## BAB[\S\s]*?)(?=\s+(?:## BAB|agar setiap orang mengetahuinya|ditetapkan di))'                  # Daftar Bab
#     },
#     'chapter': {
#         # For every chapters
#         'about': r'## (BAB [\w\s\-\/\,]+)##',                            # Nama Bab, ganti [\-\/\,] dengan semua tanda baca kecuali #
#         'part': r'(## Bagian [\S\s]*?)(?=\s+(?:## Bagian|$))',           # Daftar Bagian
#         'paragraph': r'(## Paragraf [\S\s]*?)(?=\s+(?:## Paragraf|$))',  # Daftar Paragraf
#         'article': r'(## Pasal \w+[\S\s]*?)(?=(?:##|$))'                 # Daftar Pasal
#     },
#     'part': {
#         # For every parts
#         'about': r'## (Bagian [\w\s\-\/\,]+)##'    # Nama Bagian
#     },
#     'paragraph': {
#         # For every paragraphs
#         'about': r'## (Paragraf [\w\s\-\/\,]+)##'  # Nama Paragraf
#     },
#     'article': {
#         # For every articles
#         'number': r'## Pasal (\d+\w*)',                                       # Nomor Pasal
#         'text': r'## Pasal \w+\n*([\S\s]*)',                                  # Isi Pasal
#         'definition': r'\(\d+[a-z]?\) (([A-Z][a-z]*(?:\s[A-Z][a-z]*)*) .*)',  # Daftar Definisi
#         'reference_1': r'Pasal (\d+\w*)',                                     # Pasal Referensi Jenis 1
#         'reference_2': r'Pasal (\d+\w*) sampai dengan Pasal (\d+\w*)'         # Pasal Referensi Jenis 2
#     },
#     'amendment_to': {
#         'amendment_point_1': r'(## \d+\.[\S\s]*?)(?=\s+(?:## \d+\.|## Pasal II))',  # Poin Amandemen Jenis 1
#         'amendment_point_2': r'(?<=## Pasal I)([\s\S]*?)(?=## Pasal II)',           # Poin Amandemen Jenis 2
#         'part': r'(## Bagian [\S\s]*?)(?=\s+(?:## Bagian|$))',                      # Daftar Bagian
#         'paragraph': r'(## Paragraf [\S\s]*?)(?=\s+(?:## Paragraf|$))',             # Daftar Paragraf
#         # 'amendment_articles': r'(## Pasal \d+[A-Z]?[\S\s]*?)(?=(?:##|$))',          # Pasal Amandemen
#     }
# }

In [ ]:
# def parse_base_regulation(
#         text: str,
#         id_template: str,
#         regulation_dict: dict,
#         definition_list: list,
#         article_dict: dict
# ) -> tuple[dict, list, dict]:

#     regulation_dict['content']['chapters'] = dict()

#     # Get all chapters
#     chapters = re.findall(patterns['main']['chapter'], text, re.IGNORECASE)

#     # Iterate for every chapters
#     for num, chapter in enumerate(chapters):
#         chapter_num = num + 1
#         chapter_about = re.search(patterns['chapter']['about'], chapter, re.IGNORECASE)[1].strip().upper()
#         chapter_about = re.sub(r'\n', ': ', chapter_about, flags=re.IGNORECASE)

#         regulation_dict['content']['chapters'][chapter_num] = {
#             'id': id_template.format(
#                 reg_section=encode['section']['chapter'],
#                 section_num=str(chapter_num).zfill(3),
#                 extra_section_number='00'
#             ),
#             'about': chapter_about
#         }
    
#         regulation_dict['content']['chapters'][chapter_num]['articles'] = dict()

#         # Get all parts
#         parts = re.findall(patterns['chapter']['part'], chapter.strip() + '\n', re.IGNORECASE)

#         # If the part exists
#         if parts:
#             # Iterate for every parts
#             for part in parts:
#                 # Get part about/name
#                 part_about = re.search(patterns['part']['about'], part, re.IGNORECASE)[1].strip()
#                 part_about = re.sub(r'\n', ': ', part_about, flags=re.IGNORECASE)

#                 # Get all paragraphs
#                 paragraphs = re.findall(patterns['chapter']['paragraph'], part.strip() + '\n', re.IGNORECASE)
                
#                 # If the paragraph exists
#                 if paragraphs:
#                     # Iterate for every paragraphs
#                     for paragraph in paragraphs:
#                         # Get paragraph about/name
#                         paragraph_about = re.search(patterns['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
#                         paragraph_about = re.sub(r'\n', ': ', paragraph_about, flags=re.IGNORECASE)
                        
#                         regulation_dict, definition_list, article_dict = parse_articles(
#                             text=paragraph,
#                             chapter_num=chapter_num,
#                             part_about=part_about,
#                             paragraph_about=paragraph_about,
#                             id_template=id_template,
#                             regulation_dict=regulation_dict,
#                             definition_list=definition_list,
#                             article_dict=article_dict
#                         )

#                 else:
#                     regulation_dict, definition_list, article_dict = parse_articles(
#                         text=part,
#                         chapter_num=chapter_num,
#                         part_about=part_about,
#                         paragraph_about='',
#                         id_template=id_template,
#                         regulation_dict=regulation_dict,
#                         definition_list=definition_list,
#                         article_dict=article_dict
#                     )

#         else:
#             regulation_dict, definition_list, article_dict = parse_articles(
#                 text=chapter,
#                 chapter_num=chapter_num,
#                 part_about='',
#                 paragraph_about='',
#                 id_template=id_template,
#                 regulation_dict=regulation_dict,
#                 definition_list=definition_list,
#                 article_dict=article_dict
#             )

#     return regulation_dict, definition_list, article_dict


# def parse_amendment_regulation(
#         text: str,
#         id_template: str,
#         regulation_dict: dict,
#         definition_list: list,
#         article_dict: dict,
#         amend_regulations: list
# ) -> tuple[dict, list, dict]:
    
#     # MUNGKIN INI BUAT MASALAH? COBA DULU
#     regulation_dict['content']['articles'] = dict()
    
#     # Get all amendment points
#     amendment_points = re.findall(patterns['amendment_to']['amendment_point_1'], text, re.IGNORECASE)

#     if not amendment_points:
#         amendment_points = re.search(patterns['amendment_to']['amendment_point_2'], text, re.IGNORECASE)[1].strip()
#         first_sentence = re.search(r'^.*', amendment_points, re.IGNORECASE)[0].strip()  # Dapatkan kalimat pertama
#         amendment_points = [amendment_points.replace(first_sentence, '').strip()]		# Hapus kalimat pertama
    
#     # Iterate for every points
#     for point in amendment_points:
#         # Get all parts
#         parts = re.findall(patterns['amendment_to']['part'], point.strip() + '\n', re.IGNORECASE)

#         # If the part exists
#         if parts:
#             # Iterate for every parts
#             for part in parts:
#                 # Get part about/name
#                 part_about = re.search(patterns['part']['about'], part, re.IGNORECASE)[1].strip()
#                 part_about = re.sub(r'\n', ': ', part_about, flags=re.IGNORECASE)

#                 # Get all paragraphs
#                 paragraphs = re.findall(patterns['chapter']['paragraph'], part.strip() + '\n', re.IGNORECASE)
                
#                 # If the paragraph exists
#                 if paragraphs:
#                     # Iterate for every paragraphs
#                     for paragraph in paragraphs:
#                         # Get paragraph about/name
#                         paragraph_about = re.search(patterns['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
#                         paragraph_about = re.sub(r'\n', ': ', paragraph_about, flags=re.IGNORECASE)
                        
#                         regulation_dict, definition_list, article_dict = parse_articles(
#                             text=paragraph,
#                             chapter_num=0,
#                             part_about=part_about,
#                             paragraph_about=paragraph_about,
#                             id_template=id_template,
#                             regulation_dict=regulation_dict,
#                             definition_list=definition_list,
#                             article_dict=article_dict,
#                             amendment=True,
#                             amend_regulations=amend_regulations
#                         )

#                 else:
#                     regulation_dict, definition_list, article_dict = parse_articles(
#                         text=part,
#                         chapter_num=0,
#                         part_about=part_about,
#                         paragraph_about='',
#                         id_template=id_template,
#                         regulation_dict=regulation_dict,
#                         definition_list=definition_list,
#                         article_dict=article_dict,
#                         amendment=True,
#                         amend_regulations=amend_regulations
#                     )

#         else:
#             # Get all paragraphs
#             paragraphs = re.findall(patterns['amendment_to']['paragraph'], point.strip() + '\n', re.IGNORECASE)
            
#             # If the paragraph exists
#             if paragraphs:
#                 # Iterate for every paragraphs
#                 for paragraph in paragraphs:
#                     # Get paragraph about/name
#                     paragraph_about = re.search(patterns['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
#                     paragraph_about = re.sub(r'\n', ': ', paragraph_about, flags=re.IGNORECASE)

#                     regulation_dict, definition_list, article_dict = parse_articles(
#                         text=paragraph,
#                         chapter_num=0,
#                         part_about='',
#                         paragraph_about=paragraph_about,
#                         id_template=id_template,
#                         regulation_dict=regulation_dict,
#                         definition_list=definition_list,
#                         article_dict=article_dict,
#                         amendment=True,
#                         amend_regulations=amend_regulations
#                     )

#             else:
#                 regulation_dict, definition_list, article_dict = parse_articles(
#                 	text=point,
#                     chapter_num=0,
#                     part_about='',
#                     paragraph_about='',
#                 	id_template=id_template,
#                     regulation_dict=regulation_dict,
#                 	definition_list=definition_list,
#                     article_dict=article_dict,
#                     amendment=True,
#                     amend_regulations=amend_regulations
#                 )

#     return regulation_dict, definition_list, article_dict


# def parse_articles(
#         text: str,
#         chapter_num: int,
#         part_about: str,
#         paragraph_about: str,
#         id_template: str,
#         regulation_dict: dict,
#         definition_list: list,
#         article_dict: dict,
#         amendment: bool = False,
#         amend_regulations: list = list() 
# ) -> tuple[dict, list]:
    
#     # Get all articles
#     articles = re.findall(patterns['chapter']['article'], text, re.IGNORECASE)

#     # Iterate for every articles
#     for article in articles:
#         article_num = re.search(patterns['article']['number'], article, re.IGNORECASE)[1]
#         article_text = re.search(patterns['article']['text'], article, re.IGNORECASE)[1].strip()
#         article_text = re.sub(r'\n+', '\n', article_text)
        
#         # Get definition (Article/Pasal 1)
#         if article_num == '1':
#             definitions = re.findall(patterns['article']['definition'], article_text)
#             for index, definition_data in enumerate(definitions):
#                 definition, name = definition_data
#                 definition_list.append({
#                     'id': id_template.format(
#                         reg_section=encode['section']['definition'],
#                         section_num=str(index + 1).zfill(3),
#                         extra_section_number='00'
#                     ),
#                     'name': name.strip(),
#                     'definition': definition.strip()
#                 })
#             regulation_dict['content']['definitions'] = definition_list

#         # Define article ID
#         article_id = ''

#         if amendment:
#             # Create article ID
#             article_alphabet = re.search(r'\d+([A-Z]+)', str(article_num), re.IGNORECASE)

#             if article_alphabet:
#                 article_id = id_template.format(
#                     reg_section=encode['section']['article'],
#                     section_num=re.search(r'\d+', str(article_num), re.IGNORECASE)[0].zfill(3),
#                     extra_section_number=mapping_alphabet_to_numeric[article_alphabet[1]]
#                 )
#             else:
#                 article_id = id_template.format(
#                     reg_section=encode['section']['article'],
#                     section_num=str(article_num).zfill(3),
#                     extra_section_number='00'
#                 )

#             # Store article data
#             regulation_dict['content']['articles'][article_num] = {
#                 'id': article_id,
#                 'part': part_about,
#                 'paragraph': paragraph_about,
#                 'text': article_text
#             }

#             # Store article to article_dict
#             article_dict[article_id] = regulation_dict['content']['articles'][article_num]

#             # Mendefinisikan daftar pasasl yang diamandemen
#             regulation_dict['content']['articles'][article_num]['amend'] = list()
            
#             # Ambil daftar peraturan amend
#             for regulation_id in amend_regulations:
#                 # Buat ulang (tebak) ID Pasal di peraturan sebelum revisi
#                 other_article_id = regulation_id[:-6] + '6' + article_id[-5:]

#                 # Jika ID tersebut ada di article_dict maka ID nya pasti ada, terus simpan 
#                 if other_article_id in article_dict.keys():
#                     regulation_dict['content']['articles'][article_num]['part'] = \
#                         part_about if part_about else article_dict[other_article_id]['part']
#                     regulation_dict['content']['articles'][article_num]['paragraph'] = \
#                         paragraph_about if paragraph_about else article_dict[other_article_id]['paragraph']
#                     regulation_dict['content']['articles'][article_num]['amend'].append(other_article_id)

#         else:
#             # Create article ID
#             article_id = id_template.format(
#                 reg_section=encode['section']['article'],
#                 section_num=str(article_num).zfill(3),
#                 extra_section_number='00'
#             )

#             # Store article
#             regulation_dict['content']['chapters'][chapter_num]['articles'][article_num] = {
#                 'id': article_id,
#                 'part': part_about,
#                 'paragraph': paragraph_about,
#                 'text': article_text
#             }

#             # Store article to article_dict
#             article_dict[article_id] = regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]

#             # Get previous article ID
#             if article_num != '1':
#                 regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['previous_article'] = \
#                     id_template.format(
#                         reg_section=encode['section']['article'],
#                         section_num=str(int(article_num) - 1).zfill(3),
#                         extra_section_number='00'
#                     )
#             else:
#                 regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['previous_article'] = ''
            
#             # Get article reference to other article
#             regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['references'] = list()
#             reference_type_1 = list(set(re.findall(patterns['article']['reference_1'], article_text, re.IGNORECASE)))
#             reference_type_2 = list(set(re.findall(patterns['article']['reference_2'], article_text, re.IGNORECASE)))

#             article_references = merge_article_references(reference_type_1, reference_type_2)

#             for article_reference_num in article_references:
#                 regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['references'].append(
#                     id_template.format(
#                         reg_section=encode['section']['article'],
#                         section_num=str(article_reference_num).zfill(3),
#                         extra_section_number='00'
#                     )
#                 )
                    
#     return regulation_dict, definition_list, article_dict


# def merge_article_references(list1, list2):
#     set1 = set(int(x) for x in list1)
#     set2 = set()

#     for start, end in list2:
#         set2.update(range(int(start), int(end) + 1))

#     return sorted(set1.union(set2))

In [ ]:
# input_dir = os.path.join('data', 'markdown', 'fix', 'temp')
# json_input = os.path.join('data', 'regulation_data_modified.json')
# json_output = os.path.join('data', 'regulation_data_final.json')

# regulation_data = parse_regulations_content(
#     input_dir=input_dir,
#     json_input=json_input,
#     json_output=json_output,
#     verbose=True
# )

In [ ]:
# encode = {
#     'type': {
#         'UU': '01',
#         'PERPPU': '02',
#         'PP': '03',
#         'PERPRES': '04',
#         'PERMENKOMINFO': '05'
#     },
#     'section': {
#         'document': '1',
#         'considering': '2',
#         'observing': '3',
#         'definition': '4',
#         'chapter': '5',
#         'article': '6',
#         'section': '7',
#     }
# }

# patterns = {
#     'document': {
#         'metadata': r'^(\w+)_(\w+)_(\w+)'    # Jenis, tahun, dan nomor peraturan
#     },
#     'main': {
#         'considering': r'(?<=## menimbang)([\S\s]*?)(?=## mengingat)',                                                 # Menimbang
#         'observing': r'(?<=## mengingat)([\S\s]*?)(?=(?:dengan persetujuan bersama|## memperhatikan|## memutuskan))',  # Mengingat
#         'amendment_to': r'^Perubahan',                                                                                 # Cek Peraturan Revisi
#         'chapter': r'(## BAB[\S\s]*?)(?=\s+(?:## BAB|agar setiap orang mengetahuinya))'                                # Daftar Bab
#     },
#     'chapter': {
#         # For every chapters
#         'about': r'## (BAB [\w\s\-\/\,]+)##',                            # Nama Bab, ganti [\-\/\,] dengan semua tanda baca kecuali #
#         'part': r'(## Bagian [\S\s]*?)(?=\s+(?:## Bagian|$))',           # Daftar Bagian
#         'paragraph': r'(## Paragraf [\S\s]*?)(?=\s+(?:## Paragraf|$))',  # Daftar Paragraf
#         'article': r'(## Pasal \w+[\S\s]*?)(?=(?:##|$))'                 # Daftar Pasal
#     },
#     'part': {
#         # For every parts
#         'about': r'## (Bagian [\w\s\-\/\,]+)##'    # Nama Bagian
#     },
#     'paragraph': {
#         # For every paragraphs
#         'about': r'## (Paragraf [\w\s\-\/\,]+)##'  # Nama Paragraf
#     },
#     'article': {
#         # For every articles
#         'number': r'## Pasal (\d+\w*)',                                       # Nomor Pasal
#         'text': r'## Pasal \w+\n*([\S\s]*)',                                  # Isi Pasal
#         'definition': r'\(\d+[a-z]?\) (([A-Z][a-z]*(?:\s[A-Z][a-z]*)*) .*)',  # Daftar Definisi
#         'reference_1': r'Pasal (\d+\w*)',                                     # Pasal Referensi Jenis 1
#         'reference_2': r'Pasal (\d+\w*) sampai dengan Pasal (\d+\w*)'         # Pasal Referensi Jenis 2
#     },
#     'amendment_to': {
#         'amendment_point_1': r'(## \d+\.[\S\s]*?)(?=\s+(?:## \d+\.|## Pasal II))',  # Poin Amandemen Jenis 1
#         'amendment_point_2': r'(?<=## Pasal I)([\s\S]*?)(?=## Pasal II)',           # Poin Amandemen Jenis 2
#         'amendment_articles': r'(## Pasal \d+[A-Z]?[\S\s]*?)(?=(?:##|$))',          # Pasal Amandemen
#         'part': r'(## Bagian [\S\s]*?)(?=\s+(?:## Bagian|$))',                      # Daftar Bagian
#         'paragraph': r'(## Paragraf [\S\s]*?)(?=\s+(?:## Paragraf|$))',             # Daftar Paragraf
#         # 'article': r'(## Pasal \w+[\S\s]*?)(?=(?:##|$))'                            # Daftar Pasal
#     }
# }

In [ ]:
# file = 'PERMENKOMINFO_2020_002.md'  # Tanpa Poin Revisi
# file = 'PERMENKOMINFO_2021_010.md'  # Tanpa Poin Revisi
# file = 'PERMENKOMINFO_2007_038.md'  # Dengan Poin Revisi Lebih dari 1 Pasal
# file = 'PERMENKOMINFO_2021_001.md'  # Dengan Poin Revisi Paragraf
# with open(f'data/markdown/fix/amendment/{file}', 'r', encoding='utf8') as file:
# 	text = file.read()

# amendment_points = re.findall(r'(## \d+\.[\S\s]*?)(?=\s+(?:## \d+\.|## Pasal II))', text, re.IGNORECASE)
# if not amendment_points:
# 	amendment_points = re.search(r'(?<=## Pasal I)([\s\S]*?)(?=## Pasal II)', text, re.IGNORECASE)[1].strip()
# 	first_sentence = re.search(r'^.*', amendment_points, re.IGNORECASE)[0].strip()  # Dapatkan kalimat pertama
# 	amendment_points = [amendment_points.replace(first_sentence, '').strip()]		   # Hapus kalimat pertama

# for point in amendment_points:
# 	# Get all parts
# 	parts = re.findall(patterns['amendment_to']['part'], point.strip() + '\n', re.IGNORECASE)

# 	# If the part exists
# 	if parts:
# 		# Iterate for every parts
# 		for part in parts:
# 			# Get part about/name
# 			part_about = re.search(patterns['part']['about'], part, re.IGNORECASE)[1].strip()
# 			part_about = re.sub(r'\n', ': ', part_about, flags=re.IGNORECASE)

# 			# Get all paragraphs
# 			paragraphs = re.findall(patterns['chapter']['paragraph'], part.strip() + '\n', re.IGNORECASE)
			
# 			# If the paragraph exists
# 			if paragraphs:
# 				# Iterate for every paragraphs
# 				for paragraph in paragraphs:
# 					# Get paragraph about/name
# 					paragraph_about = re.search(patterns['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
# 					paragraph_about = re.sub(r'\n', ': ', paragraph_about, flags=re.IGNORECASE)
# 					pass
# 					# regulation_dict, definition_list, article_id_list = parse_articles(
# 					# 	text=paragraph,
# 					# 	chapter_num=chapter_num,
# 					# 	part_about=part_about,
# 					# 	paragraph_about=paragraph_about,
# 					# 	regulation_dict=regulation_dict,
# 					# 	definition_list=definition_list,
# 					# 	article_id_list=article_id_list,
# 					# 	id_template=id_template
# 					# )

# 			else:
# 				pass
# 				# regulation_dict, definition_list, article_id_list = parse_articles(
# 				# 	text=part,
# 				# 	chapter_num=chapter_num,
# 				# 	part_about=part_about,
# 				# 	paragraph_about='',
# 				# 	regulation_dict=regulation_dict,
# 				# 	definition_list=definition_list,
# 				# 	article_id_list=article_id_list,
# 				# 	id_template=id_template
# 				# )

# 	else:
# 		# Get all paragraphs
# 		paragraphs = re.findall(patterns['amendment_to']['paragraph'], point.strip() + '\n', re.IGNORECASE)
		
# 		# If the paragraph exists
# 		if paragraphs:
# 			# Iterate for every paragraphs
# 			for paragraph in paragraphs:
# 				# Get paragraph about/name
# 				paragraph_about = re.search(patterns['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
# 				paragraph_about = re.sub(r'\n', ': ', paragraph_about, flags=re.IGNORECASE)
# 				print(paragraph_about)
# 				print(paragraph)
# 				print('=' * 100)

# 				# regulation_dict, definition_list, article_id_list = parse_articles(
# 				# 	text=paragraph,
# 				# 	chapter_num=chapter_num,
# 				# 	part_about=part_about,
# 				# 	paragraph_about=paragraph_about,
# 				# 	regulation_dict=regulation_dict,
# 				# 	definition_list=definition_list,
# 				# 	article_id_list=article_id_list,
# 				# 	id_template=id_template
# 				# )

# 		else:
# 			print(point)
# 			print('=' * 100)
# 			# regulation_dict, definition_list, article_id_list = parse_articles(
# 			# 	text=part,
# 			# 	chapter_num=chapter_num,
# 			# 	part_about=part_about,
# 			# 	paragraph_about='',
# 			# 	regulation_dict=regulation_dict,
# 			# 	definition_list=definition_list,
# 			# 	article_id_list=article_id_list,
# 			# 	id_template=id_template
# 			# )
			

In [ ]:
# def merge_article_references(list1, list2):
#     list1_int = set(int(x) for x in list1)
#     list2_int_expanded = set()
#     for start, end in list2:
#         list2_int_expanded.update(range(int(start), int(end) + 1))
#     return sorted(list1_int.union(list2_int_expanded))


# def parse_amendment_articles(
#         text: str,
#         id_template: str,
#         regulation_dict: dict,
#         definition_list: list,
#         article_dict: dict,
#         amendment: bool = True,
#         amend_regulations: list = list() 
# ):
#     amendment_points = re.findall(r'(## \d+\.[\S\s]*?)(?=\s+(?:## \d+\.|## Pasal II))', text, re.IGNORECASE)
#     if not amendment_points:
#         amendment_points = re.search(r'(?<=## Pasal I)([\s\S]*?)(?=## Pasal II)', text, re.IGNORECASE)[1].strip()
#         first_sentence = re.search(r'^.*', amendment_points, re.IGNORECASE)[0].strip()  # Dapatkan kalimat pertama
#         amendment_points = [amendment_points.replace(first_sentence, '').strip()]		   # Hapus kalimat pertama
    
#     for point in amendment_points:
#         # Get all parts
#         parts = re.findall(patterns['amendment_to']['part'], point.strip() + '\n', re.IGNORECASE)

#         # If the part exists
#         if parts:
#             # Iterate for every parts
#             for part in parts:
#                 # Get part about/name
#                 part_about = re.search(patterns['part']['about'], part, re.IGNORECASE)[1].strip()
#                 part_about = re.sub(r'\n', ': ', part_about, flags=re.IGNORECASE)

#                 # Get all paragraphs
#                 paragraphs = re.findall(patterns['chapter']['paragraph'], part.strip() + '\n', re.IGNORECASE)
                
#                 # If the paragraph exists
#                 if paragraphs:
#                     # Iterate for every paragraphs
#                     for paragraph in paragraphs:
#                         # Get paragraph about/name
#                         paragraph_about = re.search(patterns['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
#                         paragraph_about = re.sub(r'\n', ': ', paragraph_about, flags=re.IGNORECASE)
                        
#                         regulation_dict, definition_list, article_dict = parse_articles(
#                             text=paragraph,
#                             chapter_num=0,
#                             part_about=part_about,
#                             paragraph_about=paragraph_about,
#                             id_template=id_template,
#                             regulation_dict=regulation_dict,
#                             definition_list=definition_list,
#                             article_dict=article_dict,
#                             amendment=amendment,
#                             amend_regulations=amend_regulations
#                         )

#                 else:
#                     regulation_dict, definition_list, article_dict = parse_articles(
#                         text=part,
#                         chapter_num=0,
#                         part_about=part_about,
#                         paragraph_about='',
#                         id_template=id_template,
#                         regulation_dict=regulation_dict,
#                         definition_list=definition_list,
#                         article_dict=article_dict,
#                         amendment=amendment,
#                         amend_regulations=amend_regulations
#                     )

#         else:
#             # Get all paragraphs
#             paragraphs = re.findall(patterns['amendment_to']['paragraph'], point.strip() + '\n', re.IGNORECASE)
            
#             # If the paragraph exists
#             if paragraphs:
#                 # Iterate for every paragraphs
#                 for paragraph in paragraphs:
#                     # Get paragraph about/name
#                     paragraph_about = re.search(patterns['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
#                     paragraph_about = re.sub(r'\n', ': ', paragraph_about, flags=re.IGNORECASE)

#                     regulation_dict, definition_list, article_dict = parse_articles(
#                         text=paragraph,
#                         chapter_num=0,
#                         part_about='',
#                         paragraph_about=paragraph_about,
#                         id_template=id_template,
#                         regulation_dict=regulation_dict,
#                         definition_list=definition_list,
#                         article_dict=article_dict,
#                         amendment=amendment,
#                         amend_regulations=amend_regulations
#                     )

#             else:
#                 regulation_dict, definition_list, article_dict = parse_articles(
#                 	text=point,
#                 	chapter_num=0,
#                 	part_about='',
#                 	paragraph_about='',
#                 	id_template=id_template,
#                 	regulation_dict=regulation_dict,
#                 	definition_list=definition_list,
#                 	article_dict=article_dict,
#                     amendment=amendment,
#                     amend_regulations=amend_regulations
#                 )

#     return regulation_dict, definition_list, article_dict


# def parse_articles(
#         text: str,
#         chapter_num: int,
#         part_about: str,
#         paragraph_about: str,
#         id_template: str,
#         regulation_dict: dict,
#         definition_list: list,
#         article_dict: dict,
#         amendment: bool = False,
#         amend_regulations: list = list() 
# ) -> tuple[dict, list]:
    
#     mapping_alphabet_to_numeric = {chr(i): f"{i - 64:02}" for i in range(65, 91)}  # A-Z -> 01-26
    
#     # Get all articles
#     articles = re.findall(patterns['chapter']['article'], text, re.IGNORECASE)

#     # Iterate for every articles
#     for article in articles:
#         article_num = re.search(patterns['article']['number'], article, re.IGNORECASE)[1]
#         article_text = re.search(patterns['article']['text'], article, re.IGNORECASE)[1].strip()
#         article_text = re.sub(r'\n+', '\n', article_text)
#         # print(f'Pasal {article_num}')
#         # print(article_text, '\n')
        
#         # Get definition (Article/Pasal 1)
#         if article_num == '1':
#             definitions = re.findall(patterns['article']['definition'], article_text)
#             for index, definition_data in enumerate(definitions):
#                 definition, name = definition_data
#                 definition_list.append({
#                     'id': id_template.format(
#                         reg_section=encode['section']['definition'],
#                         section_num=str(index + 1).zfill(3),
#                         extra_section_number='00'
#                     ),
#                     'name': name.strip(),
#                     'definition': definition.strip()
#                 })
#             regulation_dict['content']['definitions'] = definition_list

#         article_id = ''

#         if amendment:
#             # Create article ID
#             article_alphabet = re.search(r'\d+([A-Z]+)', str(article_num), re.IGNORECASE)

#             if article_alphabet:
#                 article_id = id_template.format(
#                     reg_section=encode['section']['article'],
#                     section_num=re.search(r'\d+', str(article_num), re.IGNORECASE)[0].zfill(3),
#                     extra_section_number=mapping_alphabet_to_numeric[article_alphabet[1]]
#                 )
#             else:
#                 article_id = id_template.format(
#                     reg_section=encode['section']['article'],
#                     section_num=str(article_num).zfill(3),
#                     extra_section_number='00'
#                 )

#             # Store article
#             regulation_dict['content']['articles'][article_num] = {
#                 'id': article_id,
#                 'part': part_about,
#                 'paragraph': paragraph_about,
#                 'text': article_text
#             }

#             # Store article to article_dict
#             article_dict[article_id] = regulation_dict['content']['articles'][article_num]

#         else:
#             # Create article ID
#             article_id = id_template.format(
#                 reg_section=encode['section']['article'],
#                 section_num=str(article_num).zfill(3),
#                 extra_section_number='00'
#             )

#             # Store article
#             regulation_dict['content']['chapters'][chapter_num]['articles'][article_num] = {
#                 'id': article_id,
#                 'part': part_about,
#                 'paragraph': paragraph_about,
#                 'text': article_text
#             }

#             # Store article to article_dict
#             article_dict[article_id] = regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]

#             # Get previous article ID
#             if article_num != '1':
#                 regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['previous_article'] = \
#                     id_template.format(
#                         reg_section=encode['section']['article'],
#                         section_num=str(int(article_num) - 1).zfill(3),
#                         extra_section_number='00'
#                     )
#             else:
#                 regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['previous_article'] = ''
            
#             # Get article reference to other article
#             regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['references'] = list()
#             reference_type_1 = list(set(re.findall(patterns['article']['reference_1'], article_text, re.IGNORECASE)))
#             reference_type_2 = list(set(re.findall(patterns['article']['reference_2'], article_text, re.IGNORECASE)))

#             article_references = merge_article_references(reference_type_1, reference_type_2)

#             for article_reference_num in article_references:
#                 regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['references'].append(
#                     id_template.format(
#                         reg_section=encode['section']['article'],
#                         section_num=str(article_reference_num).zfill(3),
#                         extra_section_number='00'
#                     )
#                 )
            
#         # TODO 1: Membuat parsing khusus untuk peraturan revisi
#         # TODO 2: Menambah referensi peraturan amandemen
#         # Misal, jika pasal 5 merupakan pasal revisi, maka harus dihubungkan ke pasal yang sama di dokumen yang sebelumnya
#         # Lalu kode untuk pasal 10A, itu harus diakhiri dengan extra_section_number='01', kalau 10B maka '02', dst. 
#         # TODO 3: Membuat setiap JSON peraturan sebagai file terpisah? Karena terlalu rumit kalau jadi satu
#         if amendment:
#             regulation_dict['content']['articles'][article_num]['amend'] = list()
            
#             # Ambil daftar peraturan amend
#             for regulation_id in amend_regulations:
#                 # Buat ulang (tebak) ID Pasal di peraturan sebelum revisi
#                 other_article_id = regulation_id[:-6] + '6' + article_id[-5:]

#                 # Jika ID tersebut ada di article_dict maka ID nya pasti ada, terus simpan 
#                 if other_article_id in article_dict.keys():
#                     regulation_dict['content']['articles'][article_num]['part'] = \
#                         part_about if part_about else article_dict[other_article_id]['part']
#                     regulation_dict['content']['articles'][article_num]['paragraph'] = \
#                         paragraph_about if paragraph_about else article_dict[other_article_id]['paragraph']
#                     regulation_dict['content']['articles'][article_num]['amend'].append(other_article_id)
                    
#     return regulation_dict, definition_list, article_dict

In [ ]:
# DIR_PATH = os.path.join('data', 'markdown', 'fix', 'temp')
# article_dict = dict()
# result = list()
# files = list()

# start_time = time.time()

# for filename in os.listdir(DIR_PATH):
#     if filename.endswith(".md"):
#         files.append((os.path.join(DIR_PATH, filename), filename))

# for index, data in enumerate(files):
#     # Initialize data
#     path, filename = data
#     regulation_dict = dict()
#     definition_list = list()
#     print(f'File: {path}')

#     # Get file metadata
#     metadata = re.search(patterns['document']['metadata'], filename)
#     regulation_type = encode['type'][metadata[1]]
#     regulation_year = metadata[2]
#     regulation_num = int(metadata[3])

#     id_template = f'{regulation_year}{regulation_type}{str(regulation_num).zfill(3)}' + '{reg_section}{section_num}{extra_section_number}'

#     regulation_id = id_template.format(reg_section=encode['section']['document'], section_num='000', extra_section_number='00')

#     with open('data/regulation_data_modified.json') as input_file:
#         for regulation_data in json.load(input_file):
#             if regulation_data['id'] == regulation_id:
#                 regulation_dict = regulation_data

#     with open(path, 'r', encoding='utf8') as file:
#         # Read file
#         text = file.read()

#         # Define main content
#         regulation_dict['content'] = dict()

#         # Get main structure
#         regulation_dict['content']['considering'] = {
#             'id': id_template.format(reg_section=encode['section']['considering'], section_num='000', extra_section_number='00'),
#             'text': re.search(patterns['main']['considering'], text, re.IGNORECASE)[1].strip()
#         }
#         regulation_dict['content']['observing'] = {
#             'id': id_template.format(reg_section=encode['section']['observing'], section_num='000', extra_section_number='00'), 
#             'text': re.search(patterns['main']['observing'], text, re.IGNORECASE)[1].strip()
#         }

#         # If it is an amendment regulation
#         if re.search(patterns['main']['amendment_to'], regulation_dict['about'], re.IGNORECASE):
#             regulation_dict['content']['articles'] = dict()
#             regulation_dict, definition_list, article_dict = parse_amendment_articles(
#                 text=text,
#                 id_template=id_template,
#                 regulation_dict=regulation_dict,
#                 definition_list=definition_list,
#                 article_dict=article_dict,
#                 amendment=True,
#                 amend_regulations=regulation_dict['status']['amend']
#             )
        
#             # text=paragraph,
#             # chapter_num=chapter_num,
#             # part_about=part_about,
#             # paragraph_about=paragraph_about,
#             # regulation_dict=regulation_dict,
#             # definition_list=definition_list,
#             # article_dict=article_dict,
#             # id_template=id_template
        
#         else:
#             # Get all chapters
#             chapters = re.findall(patterns['main']['chapter'], text, re.IGNORECASE)
#             regulation_dict['content']['chapters'] = dict()

#             # Iterate for every chapters
#             for num, chapter in enumerate(chapters):
#                 chapter_num = num + 1
#                 chapter_about = re.search(patterns['chapter']['about'], chapter, re.IGNORECASE)[1].strip().upper()
#                 chapter_about = re.sub(r'\n', ': ', chapter_about, flags=re.IGNORECASE)

#                 regulation_dict['content']['chapters'][chapter_num] = {
#                     'id': id_template.format(
#                         reg_section=encode['section']['chapter'],
#                         section_num=str(chapter_num).zfill(3),
#                         extra_section_number='00'
#                     ),
#                     'about': chapter_about
#                 }
            
#                 regulation_dict['content']['chapters'][chapter_num]['articles'] = dict()

#                 # Get all parts
#                 parts = re.findall(patterns['chapter']['part'], chapter.strip() + '\n', re.IGNORECASE)

#                 # If the part exists
#                 if parts:
#                     # Iterate for every parts
#                     for part in parts:
#                         # Get part about/name
#                         part_about = re.search(patterns['part']['about'], part, re.IGNORECASE)[1].strip()
#                         part_about = re.sub(r'\n', ': ', part_about, flags=re.IGNORECASE)

#                         # Get all paragraphs
#                         paragraphs = re.findall(patterns['chapter']['paragraph'], part.strip() + '\n', re.IGNORECASE)
                        
#                         # If the paragraph exists
#                         if paragraphs:
#                             # Iterate for every paragraphs
#                             for paragraph in paragraphs:
#                                 # Get paragraph about/name
#                                 paragraph_about = re.search(patterns['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
#                                 paragraph_about = re.sub(r'\n', ': ', paragraph_about, flags=re.IGNORECASE)
                                
#                                 regulation_dict, definition_list, article_dict = parse_articles(
#                                     text=paragraph,
#                                     chapter_num=chapter_num,
#                                     part_about=part_about,
#                                     paragraph_about=paragraph_about,
#                                     id_template=id_template,
#                                     regulation_dict=regulation_dict,
#                                     definition_list=definition_list,
#                                     article_dict=article_dict,
#                                     amendment=False,
#                                     amend_regulations=list()
#                                 )

#                         else:
#                             regulation_dict, definition_list, article_dict = parse_articles(
#                                 text=part,
#                                 chapter_num=chapter_num,
#                                 part_about=part_about,
#                                 paragraph_about='',
#                                 id_template=id_template,
#                                 regulation_dict=regulation_dict,
#                                 definition_list=definition_list,
#                                 article_dict=article_dict,
#                                 amendment=False,
#                                 amend_regulations=list()
#                             )

#                 else:
#                     regulation_dict, definition_list, article_dict = parse_articles(
#                         text=chapter,
#                         chapter_num=chapter_num,
#                         part_about='',
#                         paragraph_about='',
#                         id_template=id_template,
#                         regulation_dict=regulation_dict,
#                         definition_list=definition_list,
#                         article_dict=article_dict,
#                         amendment=False,
#                         amend_regulations=list()
#                     )
        
#         result.append(regulation_dict)

# end_time = time.time() - start_time
# display(f'{round(end_time * 1000, 2)} milisecond')

# # https://www.freecodecamp.org/news/how-to-pretty-print-json-in-python/
# # https://www.geeksforgeeks.org/how-to-convert-python-dictionary-to-json/
# # Convert the data to a JSON formatted string with 4 spaces of indentation
# with open("output_new_new.json", "w") as outfile: 
#     json.dump(result, outfile, indent=4)
#     output_json_str = json.dumps(result, indent=4)
#     # Print the pretty-printed JSON string
#     # print(output_json_str)
    
#     # display(len(article_dict))
#     display(article_dict)

In [ ]:
# encode = {
#     'type': {
#         'UU': '01',
#         'PERPPU': '02',
#         'PP': '03',
#         'PERPRES': '04',
#         'PERMENKOMINFO': '05'
#     },
#     'section': {
#         'document': '1',
#         'considering': '2',
#         'observing': '3',
#         'definition': '4',
#         'chapter': '5',
#         'article': '6',
#         'section': '7',
#     }
# }

# patterns = {
#     'document': {
#         'metadata': r'^(\w+)_(\w+)_(\w+)'    # Jenis, tahun, dan nomor peraturan
#     },
#     'main': {
#         'considering': r'(?<=## menimbang)([\S\s]*?)(?=## mengingat)',                                                 # Menimbang
#         'observing': r'(?<=## mengingat)([\S\s]*?)(?=(?:dengan persetujuan bersama|## memperhatikan|## memutuskan))',  # Mengingat
#         'chapter': r'(## BAB[\S\s]*?)(?=\s+(?:## BAB|agar setiap orang mengetahuinya))'                                # Daftar Bab
#     },
#     'chapter': {
#         # For every chapters
#         'about': r'## (BAB [\w\s\-\/\,]+)##',                              # Nama Bab
#         'part': r'(## Bagian [\S\s]*?)(?=\s+(?:## Bagian|$))',           # Daftar Bagian
#         'paragraph': r'(## Paragraf [\S\s]*?)(?=\s+(?:## Paragraf|$))',  # Daftar Paragraf
#         'article': r'(## Pasal \w+[\S\s]*?)(?=(?:##|$))'                 # Daftar Pasal
#     },
#     'part': {
#         # For every parts
#         'about': r'## (Bagian [\w\s\-\/\,]+)##'    # Nama Bagian
#     },
#     'paragraph': {
#         # For every paragraphs
#         'about': r'## (Paragraf [\w\s\-\/\,]+)##'  # Nama Paragraf
#     },
#     'article': {
#         # For every articles
#         'number': r'## Pasal (\d+\w*)',                                 # Nomor Pasal
#         'text': r'## Pasal \w+\n*([\S\s]*)',                            # Isi Pasal
#         'definition': r'\(\d+\) (([A-Z][a-z]*(?:\s[A-Z][a-z]*)*) .*)',  # Daftar Definisi
#         'reference_1': r'Pasal (\d+\w*)',                               # Pasal Referensi Jenis 1
#         'reference_2': r'Pasal (\d+\w*) sampai dengan Pasal (\d+\w*)'   # Pasal Referensi Jenis 2
#     }
# }

In [ ]:
# text = '''## BAB II
# PENGELOLA NAMA DOMAIN

# ## Bagian Kesatu
# Umum

# ## Pasal 5
# (1) Pengelolaan Nama Domain diselenggarakan oleh Pengelola Nama Domain.
# (2) Nama Domain terdiri atas:
# 	a. Nama Domain Tingkat Tinggi Generik;
# 	b. Nama Domain Tingkat Tinggi Indonesia;
# 	c. Nama Domain Indonesia Tingkat Kedua; dan
# 	d. Nama Domain Indonesia Tingkat Turunan.
# (3) Pengelola Nama Domain sebagaimana dimaksud pada ayat (1) terdiri atas:
# 	a. Registri Nama Domain; dan
# 	b. Registrar Nama Domain.

# ## Pasal 6
# Pengelola Nama Domain sebagaimana dimaksud dalam Pasal 5 ayat (3) dapat diselenggarakan oleh Pemerintah dan/atau masyarakat.

# ## Bagian Kedua
# Registri Nama Domain

# ## Paragraf 1
# Umum

# ## Pasal 7
# (1) Registri Nama Domain sebagaimana dimaksud dalam Pasal 5 ayat (3) huruf a melaksanakan pengelolaan Nama Domain Tingkat Tinggi Generik dan Nama Domain Tingkat Tinggi Indonesia.
# (2) Registri Nama Domain dapat memberikan kewenangan pendaftaran Nama Domain Tingkat Tinggi Generik dan Nama Domain Tingkat Tinggi Indonesia kepada Registrar Nama Domain.
# (3) Registri Nama Domain berfungsi:
# 	a. memberikan masukan terhadap rencana pengaturan Nama Domain kepada Menteri;
# 	b. melakukan pengawasan terhadap Registrar Nama Domain; dan
# 	c. menyelesaikan perselisihan Nama Domain.

# ## Pasal 8
# Registri Nama Domain sebagaimana dimaksud dalam Pasal 7 ayat (1) terdiri atas:
# a. Registri Nama Domain Tingkat Tinggi Generik; dan
# b. Registri Nama Domain Tingkat Tinggi Indonesia.

# ## Paragraf 2
# Registri Nama Domain Tingkat Tinggi Generik

# ## Pasal 9
# (1) Registri Nama Domain Tingkat Tinggi Generik sebagaimana dimaksud dalam Pasal 8 huruf a melaksanakan pengelolaan Nama Domain Tingkat Tinggi Generik.
# (2) Registri Nama Domain Tingkat Tinggi Generik sebagaimana dimaksud pada ayat (1) wajib mengikuti ketentuan pengelolaan Nama Domain Internasional serta ketentuan peraturan perundang-undangan.

# ## Pasal 10
# Registri Nama Domain Tingkat Tinggi Generik memiliki tugas:
# a. merumuskan kebijakan di bidang pengelolaan Nama Domain Tingkat Tinggi Generik;
# b. menyiapkan, mengoperasikan, dan memelihara
# infrastruktur yang dibutuhkan serta menyediakan sistem elektronik untuk pengelolaan Nama Domain Tingkat Tinggi Generik; dan
# c. menyelenggarakan pendaftaran Nama Domain Tingkat Tinggi Generik sesuai dengan ketentuan pengelolaan Nama Domain Internasional dan ketentuan peraturan perundang-undangan.

# ## Pasal 11
# Registri Nama Domain Tingkat Tinggi Generik sebagaimana dimaksud dalam Pasal 9 bertanggung jawab dalam melakukan pengelolaan, pengoperasian, dan pemeliharaan penyelenggaraan sistem elektronik Nama Domain Tingkat Tinggi Generik yang berdomisili di Indonesia.

# ## Pasal 12
# (1) Nama Domain Tingkat Tinggi Generik yang akan didaftarkan dengan menggunakan nama wilayah, geografis, budaya, dan/atau situs nasional Indonesia serta bersifat unik harus mendapat persetujuan Menteri.
# (2) Dalam memberikan persetujuan sebagaimana dimaksud pada ayat (1), Menteri melakukan koordinasi dengan instansi terkait.

# ## Paragraf 3
# Registri Nama Domain Tingkat Tinggi Indonesia

# ## Pasal 13
# Registri Nama Domain Tingkat Tinggi Indonesia sebagaimana dimaksud dalam Pasal 8 huruf b melaksanakan pengelolaan Nama Domain Tingkat Tinggi Indonesia.

# ## Pasal 14
# Registri Nama Domain Tingkat Tinggi Indonesia memiliki tugas:
# a. merumuskan kebijakan di bidang pengelolaan Nama Domain Tingkat Tinggi Indonesia paling sedikit memuat:
# 	1) ketentuan Nama Domain yang dipublikasikan; dan
# 	2) daftar Nama Domain yang dibatasi, atas pertimbangan ketentuan peraturan perundang- undangan, kepatutan yang berlaku dalam masyarakat, dan iktikad baik.
# b. menyiapkan, mengoperasikan, dan memelihara infrastruktur yang dibutuhkan serta menyediakan sistem elektronik untuk pengelolaan Nama Domain Tingkat Tinggi Indonesia;
# c. menyelenggarakan pendaftaran Nama Domain Tingkat Tinggi Indonesia sesuai dengan ketentuan peraturan perundang-undangan, kepatutan yang berlaku dalam masyarakat, dan prinsip kehati-hatian;
# d. melaksanakan seleksi Registrar Nama Domain;
# e. memberikan peringatan kepada Registrar Nama Domain jika terindikasi melakukan pelanggaran;
# f. mencabut hak operasional Registrar Nama Domain jika terbukti melakukan pelanggaran; dan
# g. melakukan pengawasan operasional dan teknis Registrar Nama Domain.

# ## Pasal 15
# Registri Nama Domain Tingkat Tinggi Indonesia wajib:
# a. menjamin sistem elektronik Registri Nama Domain Indonesia beroperasi dengan baik, stabil, aman didukung dengan layanan yang dapat diandalkan;
# b. menempatkan pusat data dan pusat pemulihan bencana di wilayah Indonesia;
# c. melakukan pengawasan terhadap Registrar Nama Domain;
# d. memfasilitasi penyelesaian perselisihan Nama Domain Indonesia;
# e. melaporkan daftar Registrar Nama Domain kepada Menteri;
# f. mengikuti ketentuan pengelolaan Nama Domain internasional dan peraturan perundang-undangan di Indonesia; dan
# g. menyampaikan laporan berkala kepada Menteri sekurang-kurangnya 1 (satu) kali dalam setahun.

# ## Pasal 16
# Registri Nama Domain Tingkat Tinggi Indonesia berwenang:
# a. menolak pendaftaran Nama Domain Tingkat Tinggi Indonesia apabila Nama Domain tersebut tidak memenuhi persyaratan;
# b. menonaktifkan sementara penggunaan Nama Domain Tingkat Tinggi Indonesia; dan
# c. menghapus Nama Domain Tingkat Tinggi Indonesia apabila pengguna Nama Domain tersebut melanggar ketentuan peraturan perundang undangan setelah ada putusan pengadilan dan/atau lembaga arbitrase yang berkekuatan hukum tetap.

# ## Pasal 17
# Registri Nama Domain Tingkat Tinggi Indonesia sebagaimana dimaksud dalam Pasal 8 huruf b bertanggung jawab dalam melakukan pengelolaan, pengoperasian, dan pemeliharaan penyelenggaraan sistem elektronik Nama Domain Tingkat Tinggi Indonesia.

# ## Pasal 18
# Dalam hal Registri Nama Domain Tingkat Tinggi Indonesia bermaksud menghentikan kegiatannya, paling lambat 3 (tiga) bulan sebelum penghentian kegiatannya wajib menyerahkan seluruh pengelolaan Nama Domain yang dikelolanya kepada Menteri dengan memperhatikan kelangsungan Nama Domain Indonesia.

# ## Bagian Ketiga
# Registrar Nama Domain

# ## Paragraf 1
# Umum

# ## Pasal 19
# (1) Registrar Nama Domain sebagaimana dimaksud dalam Pasal 5 ayat (3) huruf b melaksanakan pendaftaran Nama Domain Tingkat Kedua dan Tingkat Turunan.
# (2) Pendaftaran Nama Domain sebagaimana dimaksud pada ayat (1) dilakukan atas permohonan Pengguna Nama Domain.

# ## Pasal 20
# Registrar Nama Domain terdiri atas:
# a. Registrar Nama Domain Instansi; dan
# b. Registrar Nama Domain Selain Instansi.

# ## Paragraf 2
# Registrar Nama Domain Instansi

# ## Pasal 21
# (1) Registrar Nama Domain Instansi sebagaimana dimaksud dalam Pasal 20 huruf a melaksanakan pendaftaran Nama Domain untuk kebutuhan Instansi Penyelenggara Negara.
# (2) Nama Domain untuk kebutuhan Instansi Penyelenggara Negara sebagaimana dimaksud pada ayat (1) terdiri atas Nama Domain .go.id dan .mil.id.
# (3) Registrar Nama Domain Instansi sebagaimana dimaksud pada ayat (1) dilaksanakan oleh Menteri.
# (4) Ketentuan lebih lanjut mengenai Registrar Nama Domain Instansi sebagaimana dimaksud pada ayat (1) dan ayat (3) diatur secara terpisah dalam peraturan menteri tersendiri.

# ## Paragraf 3
# Registrar Nama Domain Selain Instansi

# ## Pasal 22
# (1) Registrar Nama Domain Selain Instansi sebagaimana dimaksud dalam Pasal 20 huruf b melakukan pendaftaran Nama Domain Tingkat Kedua dan Nama Domain Tingkat Turunan untuk pengguna komersial dan nonkomersial.
# (2) Registrar Nama Domain Selain Instansi sebagaimana dimaksud pada ayat (1) terdiri atas:
# 	a. Registrar Nama Domain dengan Registri di Indonesia; dan
# 	b. Registrar Nama Domain dengan Registri di luar Indonesia.

# ## Pasal 23
# Registrar Nama Domain dengan Registri di Indonesia menyediakan jasa pendaftaran Nama Domain yang dikelola oleh Registri Nama Domain yang berdomisili di Indonesia.

# ## Pasal 24
# Registrar Nama Domain dengan Registri di Indonesia wajib mengikuti ketentuan Nama Domain yang ditetapkan oleh Registri Nama Domain sebagaimana dimaksud dalam Pasal 14 huruf a.

# ## Pasal 25
# Registrar Nama Domain dengan Registri di Indonesia berhak memperoleh pendapatan dengan memungut biaya pendaftaran dan/atau penggunaan Nama Domain dari Pengguna Nama Domain.

# ## Pasal 26
# Registrar Nama Domain dengan Registri di Indonesia dalam memberikan layanannya tidak bertanggung jawab terhadap segala implikasi hukum yang berkenaan dengan Nama Domain, kecuali yang diakibatkan karena kelalaiannya.

# ## Pasal 27
# (1) Dalam hal Registrar Nama Domain bermaksud menghentikan kegiatannya dan/atau tidak bisa melanjutkan kegiatannya, paling lambat 3 (tiga) bulan sebelum penghentian kegiatan, Registrar Nama Domain wajib menginformasikan kepada Registri Nama Domain dimana registar dimaksud terdaftar dan Pengguna Nama Domain.
# (2) Dalam hal Registrar Nama Domain habis masa berlaku, dicabut haknya, atau dalam keadaan memaksa, maka Registri Nama Domain berhak mengalihkan seluruh pengelolaan Nama Domain kepada Registrar Nama Domain lain yang dipilih oleh Pengguna Nama Domain.

# ## Pasal 28
# (1) Registrar Nama Domain dengan Registri di luar Indonesia menyediakan jasa pendaftaran Nama Domain yang dikelola oleh Registri Nama Domain yang berdomisili di luar Indonesia.
# (2) Registrar Nama Domain dengan Registri di luar Indonesia wajib mengikuti ketentuan Nama Domain yang ditetapkan oleh Registrinya sepanjang tidak bertentangan dengan ketentuan peraturan perundang- undangan.

# ## Pasal 29
# Registrar Nama Domain dengan Registri di luar Indonesia wajib mengikuti ketentuan sebagai pengelola Nama Domain sebagaimana diatur dalam Peraturan Menteri ini.
# '''

# parts = re.findall(r'(## Bagian [\S\s]*?)(?=\s+(?:## Bagian|$))', text, re.IGNORECASE)
# print(len(parts))
# display(parts)

In [ ]:
# def merge_article_references(list1, list2):
#     list1_int = set(int(x) for x in list1)
#     list2_int_expanded = set()
#     for start, end in list2:
#         list2_int_expanded.update(range(int(start), int(end) + 1))
#     return sorted(list1_int.union(list2_int_expanded))


# def parse_articles(
#         text: str,
#         chapter_num: int,
#         part_about: str,
#         paragraph_about: str,
#         regulation_dict: dict,
#         definition_list: list,
#         id_template: str
# ) -> tuple[dict, list]:
    
#     # Get all articles
#     articles = re.findall(patterns['chapter']['article'], text, re.IGNORECASE)

#     # Iterate for every articles
#     for article in articles:
#         article_num = int(re.search(patterns['article']['number'], article, re.IGNORECASE)[1])
#         article_text = re.search(patterns['article']['text'], article, re.IGNORECASE)[1].strip()
#         article_text = re.sub(r'\n+', '\n', article_text)
#         # print(f'Pasal {article_num}')
#         # print(article_text, '\n')
        
#         # Get definition (Article/Pasal 1)
#         if article_num == 1:
#             definitions = re.findall(patterns['article']['definition'], article_text)
#             for index, definition_data in enumerate(definitions):
#                 definition, name = definition_data
#                 definition_list.append({
#                     'id': id_template.format(
#                         reg_section=encode['section']['definition'],
#                         section_num=str(index + 1).zfill(3),
#                         extra_section_number='00'
#                     ),
#                     'name': name.strip(),
#                     'definition': definition.strip()
#                 })
#             regulation_dict['content']['definitions'] = definition_list
        
#         # Store article
#         regulation_dict['content']['chapters'][chapter_num]['articles'][article_num] = {
#             'id': id_template.format(
#                 reg_section=encode['section']['article'],
#                 section_num=str(article_num).zfill(3),
#                 extra_section_number='00'
#             ),
#             'part': part_about,
#             'paragraph': paragraph_about,
#             'text': article_text
#         }

#         # Get article reference to other article
#         regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['references'] = list()
#         reference_type_1 = list(set(re.findall(patterns['article']['reference_1'], article_text, re.IGNORECASE)))
#         reference_type_2 = list(set(re.findall(patterns['article']['reference_2'], article_text, re.IGNORECASE)))

#         article_references = merge_article_references(reference_type_1, reference_type_2)

#         for article_reference_num in article_references:
#             regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['references'].append(
#                 id_template.format(
#                     reg_section=encode['section']['article'],
#                     section_num=str(article_reference_num).zfill(3),
#                     extra_section_number='00'
#                 )
#             )
        
#         # TODO 1: Membuat parsing khusus untuk peraturan revisi
#         # TODO 2: Menambah referensi peraturan amandemen
#         # Misal, jika pasal 5 merupakan pasal revisi, maka harus dihubungkan ke pasal yang sama di dokumen yang sebelumnya
#         # Lalu kode untuk pasal 10A, itu harus diakhiri dengan extra_section_number='01', kalau 10B maka '02', dst. 
#         # TODO 3: Membuat setiap JSON peraturan sebagai file terpisah? Karena terlalu rumit kalau jadi satu
#         regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['amend'] = list()
    
#     return regulation_dict, definition_list

In [ ]:
# DIR_PATH = os.path.join('data', 'markdown', 'fix', 'temp')
# result = list()
# files = list()

# start_time = time.time()

# for filename in os.listdir(DIR_PATH):
#     if filename.endswith(".md"):
#         files.append((os.path.join(DIR_PATH, filename), filename))

# for index, data in enumerate(files):
#     # Initialize data
#     path, filename = data
#     regulation_dict = dict()
#     definition_list = list()
#     # section_index = 0

#     # # Just for filtering
#     # if filename not in  ['PERMENKOMINFO_002_2016.md']:
#     # # if filename not in  ['UU_NO_11_2008.md', 'PP_NO_71_2019.md', 'UU_NO_27_2022.md']:
#     # # if filename not in  ['UU_NO_11_2008.md']:
#     #     continue
#     print(f'File: {path}')

#     # Get file metadata
#     metadata = re.search(patterns['document']['metadata'], filename)
#     regulation_type = encode['type'][metadata[1]]
#     regulation_year = metadata[2]
#     regulation_num = int(metadata[3])

#     id_template = f'{regulation_year}{regulation_type}{str(regulation_num).zfill(3)}' + '{reg_section}{section_num}{extra_section_number}'

#     regulation_id = id_template.format(reg_section=encode['section']['document'], section_num='000', extra_section_number='00')

#     with open('data/regulation_data_modified.json') as input_file:
#         for regulation_data in json.load(input_file):
#             if regulation_data['id'] == regulation_id:
#                 regulation_dict = regulation_data

#     with open(path, 'r', encoding='utf8') as file:
#         # Read file
#         text = file.read()

#         # Define main content
#         regulation_dict['content'] = dict()

#         # Get main structure
#         regulation_dict['content']['considering'] = {
#             'id': id_template.format(reg_section=encode['section']['considering'], section_num='000', extra_section_number='00'),
#             'text': re.search(patterns['main']['considering'], text, re.IGNORECASE)[1].strip()
#         }
#         regulation_dict['content']['observing'] = {
#             'id': id_template.format(reg_section=encode['section']['observing'], section_num='000', extra_section_number='00'), 
#             'text': re.search(patterns['main']['observing'], text, re.IGNORECASE)[1].strip()
#         }

#         # Get all chapters
#         chapters = re.findall(patterns['main']['chapter'], text, re.IGNORECASE)
#         regulation_dict['content']['chapters'] = dict()

#         # Iterate for every chapters
#         for num, chapter in enumerate(chapters):
#             # if num == 1:
#             #     print(chapter)
#             chapter_num = num + 1
#             chapter_about = re.search(patterns['chapter']['about'], chapter, re.IGNORECASE)[1].strip().upper()
#             chapter_about = re.sub(r'\n', ': ', chapter_about, flags=re.IGNORECASE)

#             regulation_dict['content']['chapters'][chapter_num] = {
#                 'id': id_template.format(
#                     reg_section=encode['section']['chapter'],
#                     section_num=str(chapter_num).zfill(3),
#                     extra_section_number='00'
#                 ),
#                 'about': chapter_about
#             }
        
#             regulation_dict['content']['chapters'][chapter_num]['articles'] = dict()

#             # Get all parts
#             parts = re.findall(patterns['chapter']['part'], chapter.strip() + '\n', re.IGNORECASE)

#             # If the part exists
#             if parts:
#                 # Iterate for every parts
#                 for part in parts:
#                     # Get part about/name
#                     part_about = re.search(patterns['part']['about'], part, re.IGNORECASE)[1].strip()
#                     part_about = re.sub(r'\n', ': ', part_about, flags=re.IGNORECASE)

#                     # Get all paragraphs
#                     paragraphs = re.findall(patterns['chapter']['paragraph'], part.strip() + '\n', re.IGNORECASE)
                    
#                     # If the paragraph exists
#                     if paragraphs:
#                         # Iterate for every paragraphs
#                         for paragraph in paragraphs:
#                             # Get paragraph about/name
#                             paragraph_about = re.search(patterns['paragraph']['about'], paragraph, re.IGNORECASE)[1].strip()
#                             paragraph_about = re.sub(r'\n', ': ', paragraph_about, flags=re.IGNORECASE)
                            
#                             regulation_dict, definition_list = parse_articles(
#                                 text=paragraph,
#                                 chapter_num=chapter_num,
#                                 part_about=part_about,
#                                 paragraph_about=paragraph_about,
#                                 regulation_dict=regulation_dict,
#                                 definition_list=definition_list,
#                                 id_template=id_template
#                             )

#                     else:
#                         regulation_dict, definition_list = parse_articles(
#                             text=part,
#                             chapter_num=chapter_num,
#                             part_about=part_about,
#                             paragraph_about='',
#                             regulation_dict=regulation_dict,
#                             definition_list=definition_list,
#                             id_template=id_template
#                         )

#             else:
#                 regulation_dict, definition_list = parse_articles(
#                     text=chapter,
#                     chapter_num=chapter_num,
#                     part_about='',
#                     paragraph_about='',
#                     regulation_dict=regulation_dict,
#                     definition_list=definition_list,
#                     id_template=id_template
#                 )
        
#         result.append(regulation_dict)

# end_time = time.time() - start_time
# display(f'{round(end_time * 1000, 2)} milisecond')

# # https://www.freecodecamp.org/news/how-to-pretty-print-json-in-python/
# # https://www.geeksforgeeks.org/how-to-convert-python-dictionary-to-json/
# # Convert the data to a JSON formatted string with 4 spaces of indentation
# with open("output_new_new.json", "w") as outfile: 
#     json.dump(result, outfile, indent=4)
#     output_json_str = json.dumps(result, indent=4)
#     # Print the pretty-printed JSON string
#     print(output_json_str)

In [ ]:
            # # Get all parts
            # parts = re.findall(patterns['chapter']['part'], chapter, re.IGNORECASE)
            # if parts:
            #     for part in parts:
            #         # Get all paragraphs
            #         paragraphs = re.findall(patterns['chapter']['paragraph'], part, re.IGNORECASE)
            # else:
            #     pass


            # def parse_articles(text: str):

In [ ]:
# encode = {
#     'type': {
#         'UU': '01',
#         'PERPPU': '02',
#         'PP': '03',
#         'PERPRES': '04',
#         'PERMENKOMINFO': '05'
#     },
#     'section': {
#         'document': '1',
#         'considering': '2',
#         'observing': '3',
#         'definition': '4',
#         'chapter': '5',
#         'article': '6',
#         'section': '7',
#     }
# }

# patterns = {
#     'document': {
#         'metadata': r'^(\w+)_(\w+)_(\w+)'  # Jenis, tahun, dan nomor peraturan
#     },
#     'main': {
#         'considering': r'(?<=## menimbang)([\S\s]*?)(?=## mengingat)',  # Menimbang
#         'observing': r'(?<=## mengingat)([\S\s]*?)(?=(?:dengan persetujuan bersama|## memperhatikan|## memutuskan))',  # Mengingat
#         'chapter': r'(## BAB[\S\s]*?)(?=\s+(?:## BAB|agar setiap orang mengetahuinya))',  # Daftar Bab
#     },
#     'chapter': {
#         # For every chapter
#         'about': r'## BAB [IVXLCDM]+\s([\w\s]+)##',  # Nama Bab
#         'article': r'(## Pasal \w+[\S\s]*?)(?=(?:##|$))'  # Daftar Pasal [GANTI \d jadi \w karena ada Pasal 51A]
#     },
#     'article': {
#         # For every article
#         'number': r'## Pasal (\w+)',  # Nomor Pasal
#         'text': r'## Pasal \w+\n*([\S\s]*)',  # Isi Pasal
#         'definition': r'\(\d+\) (([A-Z][a-z]*(?:\s[A-Z][a-z]*)*) .*)',  # Daftar Definisi
#         # 'definition': r'\(\d+\) ((.*)(?:\badalah\b) .*)',  # Daftar Definisi
#         # \(\d+\) (([A-Z][a-z]*(?:\s[A-Z][a-z]*)*) .*)
#         # \(\d+\) (([\w\s]+) (?:yang selanjutnya|adalah).*)
#         # 'section': r'-\s*(\(\d\)) ([\S\s]*?)(?=(?:- (\(\d+\))|$))',  # Daftar Ayat (Kalau mau motong sampai per ayat, tapi ngga sempurna)
#         'reference_1': r'Pasal (\w+)',  # Pasal Referensi Jenis 1
#         'reference_2': r'Pasal (\w+) sampai dengan Pasal (\w+)',  # Pasal Referensi Jenis 2
#     }
# }

In [ ]:
# def merge_article_references(list1, list2):
#     list1_int = set(int(x) for x in list1)
#     list2_int_expanded = set()
#     for start, end in list2:
#         list2_int_expanded.update(range(int(start), int(end) + 1))
#     return sorted(list1_int.union(list2_int_expanded))

In [ ]:
# DIR_PATH = os.path.join('output', 'output')
# result = list()
# files = list()

# start_time = time.time()

# for filename in os.listdir(DIR_PATH):
#     if filename.endswith(".md"):
#         files.append((os.path.join(DIR_PATH, filename), filename))

# for index, data in enumerate(files[:5]):
#     # TODO: Tes 5 peraturan, peraturannya ada yang masih tidak lengkap isinya, peraturan revisi tidak di parsing dengan baik
#     # Initialize data
#     path, filename = data
#     regulation_dict = dict()
#     definition_list = list()
#     # section_index = 0

#     # # Just for filtering
#     # if filename not in  ['PERMENKOMINFO_002_2016.md']:
#     # # if filename not in  ['UU_NO_11_2008.md', 'PP_NO_71_2019.md', 'UU_NO_27_2022.md']:
#     # # if filename not in  ['UU_NO_11_2008.md']:
#     #     continue
#     print(f'File: {path}')

#     # Get file metadata
#     metadata = re.search(patterns['document']['metadata'], filename)
#     regulation_type = encode['type'][metadata[1]]
#     regulation_num = int(metadata[2])
#     regulation_year = metadata[3]

#     id_template = f'{regulation_year}{regulation_type}{str(regulation_num).zfill(3)}' + '{reg_section}{section_num}'

#     regulation_id = id_template.format(reg_section=encode['section']['document'], section_num='000')
#     with open('data/regulation_data_modified.json') as input_file:
#         for regulation_data in json.load(input_file):
#             if regulation_data['id'] == regulation_id:
#                 regulation_dict = regulation_data

#     with open(path, 'r', encoding='utf8') as file:
#         # Read file
#         text = file.read()

#         # Define main content
#         regulation_dict['content'] = dict()

#         # Get main structure
#         regulation_dict['content']['considering'] = {
#             'id': id_template.format(reg_section=encode['section']['considering'], section_num='000'),
#             'text': re.search(patterns['main']['considering'], text, re.IGNORECASE)[1].strip()
#         }
#         regulation_dict['content']['observing'] = {
#             'id': id_template.format(reg_section=encode['section']['observing'], section_num='000'), 
#             'text': re.search(patterns['main']['observing'], text, re.IGNORECASE)[1].strip()
#         }

#         # Get all chapters
#         chapters = re.findall(patterns['main']['chapter'], text, re.IGNORECASE)
#         regulation_dict['content']['chapters'] = dict()

#         # Iterate for every chapters
#         for num, chapter in enumerate(chapters):
#             chapter_num = num + 1
#             regulation_dict['content']['chapters'][chapter_num] = {
#                 'id': id_template.format(reg_section=encode['section']['chapter'], section_num=str(chapter_num).zfill(3)),
#                 'about': re.search(patterns['chapter']['about'], chapter, re.IGNORECASE)[1].strip()
#             }

#             # Get all articles
#             articles = re.findall(patterns['chapter']['article'], chapter, re.IGNORECASE)
#             regulation_dict['content']['chapters'][chapter_num]['articles'] = dict()

#             # Iterate for every articles
#             for article in articles:
#                 article_num = int(re.search(patterns['article']['number'], article, re.IGNORECASE)[1])
#                 article_text = re.search(patterns['article']['text'], article, re.IGNORECASE)[1].strip()
#                 article_text = re.sub(r'\n+', '\n', article_text)
                
#                 # Get definition (special article 1)
#                 if article_num == 1:
#                     definitions = re.findall(patterns['article']['definition'], article_text)
#                     for index, definition_data in enumerate(definitions):
#                         definition, name = definition_data
#                         definition_list.append({
#                             'id': id_template.format(reg_section=encode['section']['definition'], section_num=str(index + 1).zfill(3)),
#                             'name': name.strip(),
#                             'definition': definition.strip()
#                         })
#                     regulation_dict['content']['definitions'] = definition_list
                
#                 # Store article
#                 regulation_dict['content']['chapters'][chapter_num]['articles'][article_num] = {
#                     'id': id_template.format(reg_section=encode['section']['article'], section_num=str(article_num).zfill(3)),
#                     'text': article_text
#                 }

#                 # Get article reference to other article
#                 regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['references'] = list()
#                 reference_type_1 = list(set(re.findall(patterns['article']['reference_1'], article_text, re.IGNORECASE)))
#                 reference_type_2 = list(set(re.findall(patterns['article']['reference_2'], article_text, re.IGNORECASE)))

#                 article_references = merge_article_references(reference_type_1, reference_type_2)

#                 for article_reference_num in article_references:
#                     regulation_dict['content']['chapters'][chapter_num]['articles'][article_num]['references'].append(
#                         id_template.format(reg_section=encode['section']['article'], section_num=str(article_reference_num).zfill(3))
#                     )
        
#         result.append(regulation_dict)

# end_time = time.time() - start_time
# display(f'{round(end_time * 1000, 2)} milisecond')

# # https://www.freecodecamp.org/news/how-to-pretty-print-json-in-python/
# # https://www.geeksforgeeks.org/how-to-convert-python-dictionary-to-json/
# # Convert the data to a JSON formatted string with 4 spaces of indentation
# with open("output_new.json", "w") as outfile: 
#     json.dump(result, outfile, indent=4)
#     output_json_str = json.dumps(result, indent=4)
#     # Print the pretty-printed JSON string
#     print(output_json_str)

In [ ]:
# with open('data/regulation_data_modified.json') as input_file:
#     for regulation_data in json.load(input_file):
#         if regulation_data['id'] == '1999010361000':
#             display(regulation_data)

In [ ]:
# test = {
#     'key1': 'value1',
#     'key2': 'value2',
#     'key3': 'value3'
# }

# for i in test.values():
#     print(i)

# print(list(test.values()))
# print(type(list(test.values())))
# # print(test.items())

In [ ]:
# encode = {
#     'type': {
#         'UU': '01',
#         'PERPPU': '02',
#         'PP': '03',
#         'PERPRES': '04',
#         'PERMENKOMINFO': '05'
#     },
#     'section': {
#         'document': '1',
#         'considering': '2',
#         'observing': '3',
#         'definition': '4',
#         'chapter': '5',
#         'article': '6',
#         'section': '7',
#     }
# }

# patterns = {
#     'document': {
#         'metadata': r'^(.*)_NO_(\d+)_(\d+)'  # Jenis, nomor, dan tahun peraturan
#     },
#     'stopword': {
#         'hukumonline': r'(www.hukumonline.com)',  # Header/footer www.hukumonline.com
#         'image': r'(<!-- image -->)',  # Tanda image
#     },
#     'main': {
#         # 'name': r'([\S\s]*?)(?=## DENGAN RAHMAT TUHAN YANG MAHA ESA)',  # Nama Peraturan
#         'about': r'(?<=TENTANG)([\S\s]*?)(?=## DENGAN RAHMAT TUHAN YANG MAHA ESA)',  # Menetapkan (tentang)
#         'considering': r'(?<=## Menimbang:)([\S\s]*?)(?=## Mengingat)',  # Menimbang
#         'observing': r'(?<=## Mengingat:)([\S\s]*?)(?=(?:Dengan Persetujuan Bersama|## MEMUTUSKAN))',  # Mengingat
#         'chapter': r'(## BAB[\S\s]*?)(?=\s+(?:## BAB|Agar setiap orang))',  # Daftar Bab
#     },
#     'chapter': {
#         # For every chapter
#         'about': r'## BAB [IVXLCDM]+[\s\S]*?(## .*)',  # Nama Bab
#         'article': r'(## Pasal \d+[\S\s]*?)(?=(?:##|$))'  # Daftar Pasal [GANTI \d jadi \w karena ada Pasal 51A]
#     },
#     'article': {
#         # For every article
#         'number': r'## Pasal (\d+)',  # Nomor Pasal
#         'text': r'## Pasal \d+\s*([\S\s]*)',  # Isi Pasal
#         'definition': r'-\s*\d+\. (([A-Z][a-z]*(?:\s[A-Z][a-z]*)*) .*)',  # Daftar Definisi
#         'section': r'-\s*(\(\d\)) ([\S\s]*?)(?=(?:- (\(\d+\))|$))',  # Daftar Ayat
#         'reference_1': r'Pasal (\d+)',  # Pasal Referensi Jenis 1
#         'reference_2': r'Pasal (\d+) sampai dengan Pasal (\d+)',  # Pasal Referensi Jenis 2
#     }
# }

In [ ]:
# def merge_article_references(list1, list2):
#     list1_int = set(int(x) for x in list1)
#     list2_int_expanded = set()
#     for start, end in list2:
#         list2_int_expanded.update(range(int(start), int(end) + 1))
#     return sorted(list1_int.union(list2_int_expanded))

In [ ]:
# DIRECTORY_PATH = os.path.join('data', 'good')
# output = list()
# files = list()

# start_time = time.time()

# for filename in os.listdir(DIRECTORY_PATH):
#     if filename.endswith(".md"):
#         files.append((os.path.join(DIRECTORY_PATH, filename), filename))

# for index, data in enumerate(files):
#     # Initialize data
#     path, filename = data
#     regulation_dict = dict()
#     definition_dict = dict()
#     # section_index = 0

#     # Just for filtering
#     if filename not in  ['UU_NO_11_2008.md', 'PP_NO_71_2019.md', 'UU_NO_27_2022.md']:
#     # if filename not in  ['UU_NO_11_2008.md']:
#         continue
#     print(f'File: {path}')

#     # Get file metadata
#     metadata = re.search(patterns['document']['metadata'], filename)
#     regulation_type = encode['type'][metadata[1]]
#     regulation_num = metadata[2]
#     regulation_year = metadata[3]

#     id_template = f'{regulation_year}{regulation_type}{str(regulation_num).zfill(3)}' + '{reg_section}{section_num}'

#     regulation_dict['metadata'] = {
#         'id': id_template.format(reg_section=encode['section']['document'], section_num='000'),
#         'type': metadata[1],
#         'num': regulation_num,
#         'year': regulation_year
#     }
#     # print(f"Regulation ID: {regulation_dict['metadata']['id']}")

#     with open(path, 'r', encoding='utf8') as file:
#         # Read file
#         text = file.read()

#         # Remove stopword
#         text = str.strip(re.sub(patterns['stopword']['hukumonline'], '', text))
#         text = str.strip(re.sub(patterns['stopword']['image'], '', text))

#         # Get main structure
#         regulation_dict['metadata']['about'] = str.strip(re.search(patterns['main']['about'], text, re.IGNORECASE)[1]).title()
#         regulation_dict['considering'] = {
#             'id': id_template.format(reg_section=encode['section']['considering'], section_num='000'),
#             'text': str.strip(re.search(patterns['main']['considering'], text, re.IGNORECASE)[1])
#         }
#         regulation_dict['observing'] = {
#             'id': id_template.format(reg_section=encode['section']['observing'], section_num='000'), 
#             'text': str.strip(re.search(patterns['main']['observing'], text, re.IGNORECASE)[1])
#         }
#         chapters = re.findall(patterns['main']['chapter'], text, re.IGNORECASE)

#         # display(f"NAMA: {regulation_dict['name']}")
#         # print(f"MENIMBANG:\n\n{regulation_dict['considering']}")
#         # display(f"MENGINGAT: {regulation_dict['observing']}")
#         # display(f"MENETAPKAN: {regulation_dict['to_enact']}")
#         # display(chapters)

#         # Get chapter structure
#         regulation_dict['chapters'] = dict()
#         for num, chapter in enumerate(chapters):
#             chapter_num = num + 1
#             articles = re.findall(patterns['chapter']['article'], chapter, re.IGNORECASE)
#             regulation_dict['chapters'][chapter_num] = {
#                 'id': id_template.format(reg_section=encode['section']['chapter'], section_num=str(chapter_num).zfill(3)),
#                 'about': re.search(patterns['chapter']['about'], chapter, re.IGNORECASE)[1]
#             }
#             regulation_dict['chapters'][chapter_num]['articles'] = dict()

#             # print(f'\nBAB {chapter_num} {regulation_dict['chapters'][chapter_num]['about']}\n')
#             # display(articles)

#             # Get article structure
#             for article in articles:
#                 article_num = int(re.search(patterns['article']['number'], article)[1])
#                 article_text = re.search(patterns['article']['text'], article)[1]
#                 article_text = str.strip(re.sub(r'\n+', '\n', article_text))
#                 sections = re.findall(patterns['article']['section'], article_text, re.IGNORECASE)
                
#                 if article_num == 1:
#                     definition_ls = re.findall(patterns['article']['definition'], article_text)
#                     for index, definition_data in enumerate(definition_ls):
#                         definition, name = definition_data
#                         definition_dict[index + 1] = {
#                             'id': id_template.format(reg_section=encode['section']['definition'], section_num=str(index + 1).zfill(3)),
#                             'about': name,
#                             'text': str.strip(definition)
#                         }
#                     regulation_dict['definitions'] = definition_dict
#                     # display(definition_dict)
                
#                 regulation_dict['chapters'][chapter_num]['articles'][article_num] = {
#                     'id': id_template.format(reg_section=encode['section']['article'], section_num=str(article_num).zfill(3)),
#                     'text': article_text
#                 }

#                 regulation_dict['chapters'][chapter_num]['articles'][article_num]['references'] = dict()
#                 reference_type_1 = list(set(re.findall(patterns['article']['reference_1'], article_text, re.IGNORECASE)))
#                 reference_type_2 = list(set(re.findall(patterns['article']['reference_2'], article_text, re.IGNORECASE)))
                
#                 # if reference_type_1:
#                     # print(reference_type_1)
#                 # if reference_type_2: 
#                     # print(reference_type_2)
                
#                 article_references = merge_article_references(reference_type_1, reference_type_2)
#                 # print(article_references)

#                 for index, article_reference_num in enumerate(article_references):
#                     regulation_dict['chapters'][chapter_num]['articles'][article_num]['references'][index + 1] = \
#                         id_template.format(reg_section=encode['section']['article'], section_num=str(article_reference_num).zfill(3))

#                 # print(f'Pasal {article_num}')
#                 # if sections: 
#                 #     section_dict = dict()
#                 #     for section in sections:
#                 #         section_index += 1
#                 #         section_dict[section[0]] = {
#                 #             'id': id_template.format(reg_section=encode['section']['section'], section_num=str(section_index).zfill(3)),
#                 #             'text': str.strip(section[1])
#                 #         }
#                 #         regulation_dict['chapters'][chapter_num]['articles'][article_num]['sections'] = section_dict
#                 #         print(section[0], str.strip(section[1]))
#                 #     print()
#                 # else:
#                 #     print(article_text, '\n')
        
#         output.append(regulation_dict)

# end_time = time.time() - start_time
# display(f'{round(end_time * 1000, 2)} milisecond')

# # https://www.freecodecamp.org/news/how-to-pretty-print-json-in-python/
# # https://www.geeksforgeeks.org/how-to-convert-python-dictionary-to-json/
# # Convert the data to a JSON formatted string with 4 spaces of indentation
# with open("output.json", "w") as outfile: 
#     json.dump(output, outfile, indent=4)
#     output_json_str = json.dumps(output, indent=4)
#     # Print the pretty-printed JSON string
#     print(output_json_str)

In [ ]:
# pattern = r'## BAB [IVXLCDM]+[\s\S]*?\n(## .*)'
# for chapter in chapters:
#     print(re.search(pattern, chapter)[1])